# DeepRegQA - CNN for predicting quality metrics for image registrations


## Initial Setup

### Imports

In [ ]:
import tensorflow as tf
# print(tf.version.VERSION)

In [ ]:
import sys, os
# Important to assign the processing to the GPU with the most vRAM available
GPU = 0
os.environ["CUDA_VISIBLE_DEVICES"] = str(GPU) #"1"

physical_devices = tf.config.experimental.list_physical_devices('GPU')

config = tf.compat.v1.ConfigProto(device_count = {'GPU': GPU})

#Important to run this in order to not overload the GPU
config = tf.compat.v1.ConfigProto(log_device_placement=True)
config.gpu_options.per_process_gpu_memory_fraction=0.9 # don't hog all vRAM
#config.operation_timeout_in_ms=15000   # terminate on long hangs
#config.gpu_options.allow_growth = True
sess = tf.compat.v1.InteractiveSession("", config=config)
sess.close()

In [ ]:
#from __future__ import print_function
import glob
import numpy as np
import nibabel as nib
import pandas as pd
# import xarray as xr
import random
import time
import matplotlib.pyplot as plt
#import tqdm.keras
import random

### Load Previous Arrays

#### Loading Data

Use command %reset to delete all variables in case you want to clear up and load from new

[Magic commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-reset): Calling `%reset -f` after defining variables removes all variables from memory



In [ ]:
# 'BRAIN_STEM', 'SPINAL_CORD', 'PAROTID_RT', 'PAROTID_LT' 
organ = 'PAROTID_LT'

# folder = '/hepgpu9-data1/sbridger/mphys_2d_network_2021-22/slice_arrays/NEW_DATA/'+organ+'/complete_arrays/'\
# + 'COMPLETE_' + organ + '_'

train_test = 'test_even' #'test_even' train_odd

folder = '/hepgpu9-data1/sbridger/mphys_2d_network_2021-22/slice_arrays/new_registrations/{}/complete_arrays/{}_{}_'\
.format(organ, train_test, organ)

complete_img_name = folder+'multiple_patient_image_dvf_array.npy'
complete_dice_name = folder+'multiple_dice_array.npy'
complete_hd_95_name = folder+'multiple_hd_95_array.npy'
complete_hd_75_name = folder+'multiple_hd_75_array.npy'
complete_hd_50_name = folder+'multiple_hd_50_array.npy'
complete_rms_name = folder+'multiple_rms_array.npy'
complete_msd_name = folder+'multiple_msd_array.npy'

multiple_patient_image_array = np.load(complete_img_name, mmap_mode = 'r')

multiple_dice_array = np.load(complete_dice_name, mmap_mode = 'r')

multiple_hd_95_array = np.load(complete_hd_95_name, mmap_mode = 'r')

multiple_hd_75_array = np.load(complete_hd_75_name, mmap_mode = 'r')

multiple_hd_50_array = np.load(complete_hd_50_name, mmap_mode = 'r')

multiple_rms_array = np.load(complete_rms_name, mmap_mode = 'r')

multiple_msd_array = np.load(complete_msd_name, mmap_mode = 'r')


#### Data Processing

In [ ]:
metrics = {
    "dice": multiple_dice_array,
    "hd_95": multiple_hd_95_array,
    "hd_75": multiple_hd_75_array,
    "hd_50": multiple_hd_50_array,
    "msd": multiple_msd_array,
    "rms": multiple_rms_array,
}

Choose a metric

In [ ]:
metric = 'dice'

In [ ]:
try:
    raw_metric_array = metrics[metric]
    print("You have chosen: ", metric)
    #print(raw_metric_array)
except:
    print("Please choose a valid metric in the above cell (dice, hd_95, hd_75, hd_50, rms or msd)")
    

**Sort/trim the data**
- To remove all zero DICE scores, without sorting use: `metric_array = raw_metric_array[np.where(multiple_dice_array != 0)]`

In [ ]:
def normalise(array):
    min_value = np.min(array)
    max_value = np.max(array)
    n_array = (array-min_value)/(max_value-min_value)
    
    return n_array, min_value, max_value

def transformed_array(array, power, metric_name):
    print("Working with "+metric_name+":")
#     array = raw_metric_array
    print("Original min: {}    max:  {}".format(np.min(array),np.max(array)))
    
    if power!=1:
        array = array**(1/power)
        #2: Normalise metric aray
        array, min_value, max_value = normalise(array)
        print("Values after taking the {}-root and normalising: ".format(power))
        print("Min value: ", min_value, "\t Max value: ", max_value)    
    #     Repeat operation
        array = array**(1/power)
        array, min_value, max_value = normalise(array)
        print("Values after taking the {}th root and normalising: ".format(power**2))
        
    else:
        array, min_value, max_value = normalise(array)
        
    print("After normalising:")
    print("Min value: ", np.min(array), "\t Max value: ", np.max(array))
    print("Mean value: ", np.mean(array))
    
    return array

def transform_dice_array(array, metric_name):
#     array = raw_metric_array

    print("Working with "+metric_name+":")
    print("Max value no zero Dice Score: \t",np.max(array))
    print("Min value no zero Dice Score: \t",np.min(array))
    print("Mean value no zero Dice Score: \t",np.mean(array))

    zeros=0
    for i in array:
        if i == 0:
            zeros+=1

    print("Number of zero dice scores: ", str(zeros))
    array, min_value, max_value = normalise(array)

    print("After normalising:")
    print("Min value: ", np.min(array), "\t Max value: ", np.max(array))
    print("Mean value: ", np.mean(array))

    return array
    
if metric == "rms" or metric == 'msd':
    metric_array = transformed_array(raw_metric_array, 2, metric)
elif metric == "dice": 
    metric_array = transform_dice_array(raw_metric_array, metric)
else:
    metric_array = transformed_array(raw_metric_array, 1, metric)

In [ ]:
print("Our chosen metric is: ", metric)
min_value = np.min(metric_array)
max_value = np.max(metric_array)
print("Min value: ", min_value, "\t Max value: ", max_value)

### Weight Training data

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.style.use('classic')
import seaborn as sb # A data visualisation library based on matplotlib
import datetime #module to provide the current date and time for use in the file names

In [ ]:
#Produce histogram for the frequency of each value of the metric. 

n_bins = 40
n_actual_metric_hist = np.zeros(n_bins)
n_actual_metric_hist, bins_actual_metric_hist, patches_actual_metric_hist = \
    plt.hist(metric_array, density = False, bins = n_bins)#, range = (0,1))
#The greater the number of bins the more localised they are in each of the Dice Scores.
#With smaller number of bins, a bigger range of Dice Scores would correspond to the same bin (ie. lower "resolution")
# print(bins_actual_metric_hist)
# print(n_actual_metric_hist)

##### Create Array of Weights


Each bin has a weight = 1/frequency and each value in that bin has that weight.

In [ ]:
def binningWeights(data, nbins, weight_offset):
    n, bins = np.histogram(data, nbins)
    weights=[]; j=0; bin_edges=[]
    for x in data: # loop through data
        i=0
        while i in range(len(bins)): # loop through bins
#             if x==bins[i]: 
#                 bin_edges.append(x)
            if bins[i] < x <= bins[i+1]: # if data point is in bin
                j=i # save index
                i=None # exit while loop
            else:
                i=i+1
#         if j==0:
#             weights.append(1/(n[j]+weight_offset*weight_factor)) # ensure low values have less/ equal weighted freq.  
        else:
            weights.append(1/(n[j]+weight_offset)) # assign weight as 1/N for bin j
#     print(len(bin_edges))
    weight_array = np.array(weights)
    return weight_array

def binning2DWeights(data_x, data_y, nbins, weight_offset):
    n, bins_x, bins_y = np.histogram2d(data_x, data_y, nbins)
#     print(n.shape)
    weights=[]
    j_y = 0; j_x = 0
    for x, y in zip(data_x, data_y): # loop through data
        # find x bin
        i_x=0
        while i_x in range(len(bins_x)): # loop through bins
            if bins_x[i_x] <= x <= bins_x[i_x+1]: # if data point is in bin
                j_x=i_x # save index
                i_x=None # exit while loop
            else:
                i_x=i_x+1 
                
        # find y bin
        i_y=0
        while i_y in range(len(bins_y)): # loop through bins
            if bins_y[i_y] <= y <= bins_y[i_y+1]: # if data point is in bin
                j_y=i_y # save index
                i_y=None # exit while loop
            else:
                i_y=i_y+1 
        
        weights.append(1/(n[j_x,j_y]+weight_offset)) # assign weight as 1/N for bin j
#         print(1/(n[j_x,j_y]+weight_offset))
    weights_arr = np.array(weights)
#     print(weights_arr.shape)
    return weights_arr

In [ ]:
weightArray = binningWeights(metric_array, n_bins, 3)

# print(np.mean(weightArray))
weightArray /= np.mean(weightArray)
# print(np.mean(weightArray))
plt.plot(weightArray)
plt.show()
print("Mean ± std weight: {:.2f} ± {:.2f}".format(np.mean(weightArray), np.std(weightArray)))
print("Min weight {:.3f}:".format(np.min(weightArray)))
print("Max weight {:.2f}:".format(np.max(weightArray)))

# weightArray = weightArray/np.mean(weightArray)

metric_name = metric
fig, ax1 = plt.subplots(figsize=(5, 3.5))

#removes the grey background to the plot
fig.patch.set_facecolor('white')
N, Bins, Patches = plt.hist(metric_array, weights = weightArray, density = False, bins = n_bins, range = (0,1))
ax1.set_ylabel('Weighted Frequency', fontsize=18)
ax1.set_xlabel(metric_name, fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=14)
totalNumberSlices = str(metric_array.shape[0])
plt.tight_layout()
#plt.savefig(folder_arrays_load+'/frequency_histograms/'+metric+'/raw_'+metric+'_histogram_' + organ + '_' + totalNumberSlices + '_fourth_root.png')

plt.show()

print("Mean ± std weighted frequency: {:.1f} ± {:.1f}".format(np.mean(N), np.std(N)))
print("Min weighted frequency: {:.1f}".format(np.min(N)))
print("Max weighted frequency: {:.1f}".format(np.max(N)))


## Construct the Model

### Introduction

* https://keras.io/
* https://keras.io/getting-started/sequential-model-guide/ Guide to sequential model (with some examples)
* https://github.com/keras-team/keras/tree/master/examples More examples
* https://keras.io/models/sequential/ Methods of the sequential model

**Importing tensorflow**

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Dense, Activation,\
GaussianNoise, Input, concatenate, Reshape, GlobalAveragePooling2D, AveragePooling2D, UpSampling2D, ZeroPadding2D, Add
from tensorflow.keras.layers import LeakyReLU, ELU , ReLU
from tensorflow.keras.activations import relu
from tensorflow.keras.regularizers import l2
from tensorflow.keras import mixed_precision
from tensorflow.keras import initializers
#tf.keras.mixed_precision.experimental.set_policy('mixed_float16') # Mixed precision should increase processing speed

Below are 4 different models. We will use the `single_functional_feedforward_model`

In [ ]:
#Some functions
def absolute(x):
    return abs(x)

from tensorflow.keras.utils import get_custom_objects
get_custom_objects().update({'absolute': Activation(absolute)})

def relu_max(x):
    return relu(x, max_value=1)

# The different CNN models
def vector_avg_pooling_model():
    l2_reg = 0.0005
    gn = 0 #0.0005
    initializer = tf.keras.initializers.he_normal(seed=1)
    img_input = Input(shape=single_input_shape)
#     x = GaussianNoise(gn)(img_input)
#     x = Reshape((single_input_shape))(x)
    x = AveragePooling2D(pool_size=(2, 2), strides=(2, 2))(img_input)
    x = Conv2D(4, kernel_size=(3,3), input_shape=single_input_shape, strides=(1, 1), data_format = 'channels_last',\
               kernel_regularizer=l2(l2_reg), kernel_initializer=initializer)(x)
    x = ReLU()(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
    x = Dropout(0.25)(x)
#     x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    out_metric1 = Dense(1, activation='sigmoid', dtype=tf.float32, name=metric1, kernel_initializer=initializer)(x)
    out_metric2 = Dense(1, activation='sigmoid', dtype=tf.float32, name=metric2, kernel_initializer=initializer)(x)
    model = Model(inputs=img_input, outputs=[out_metric1, out_metric2])

    return model

def vector_model():
    l2_reg = 0
    gn = 0 #0.0005
    initializer = tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.07) #  he_normal(seed=1)
    img_input = Input(shape=single_input_shape)
#     x = GaussianNoise(gn)(img_input)
#     x = Reshape((single_input_shape))(x)
    x = BatchNormalization()(img_input)
    
    x = Conv2D(32, kernel_size=(3,3), input_shape=single_input_shape, strides=(1, 1),\
               kernel_regularizer=l2(l2_reg), kernel_initializer=initializer)(x)
    x = ReLU()(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
    
    x = Conv2D(64, kernel_size=(3,3), input_shape=single_input_shape, strides=(1, 1),\
               kernel_regularizer=l2(l2_reg), kernel_initializer=initializer)(img_input)
    x = ReLU()(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
    x = Flatten()(x)
    
    x = Conv2D(128, kernel_size=(3,3), input_shape=single_input_shape, strides=(2, 2),\
               kernel_regularizer=l2(l2_reg), kernel_initializer=initializer)(img_input)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
    
    x = Flatten()(x)
    x = Dropout(0.2)(x)
    
    x = Dense(8)(x)
    x = ReLU()(x)
    x = BatchNormalization()(x)
#     x = GlobalAveragePooling2D()(x)
    out_metric = Dense(1, activation='sigmoid', dtype=tf.float32, name=metric, kernel_initializer=initializer)(x)
    model = Model(inputs=img_input, outputs=[out_metric])

    return model


def vector_model_sequential():
    l2_reg = 0
    gn = 0 #0.0005
    initializer = tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.07) #he_normal(seed=1)
#     img_input = Input(shape=single_input_shape)
#     GaussianNoise(gn)(img_input)
#     Reshape((single_input_shape))
    lay1,lay2,lay3,dense = 32,64,128,32 #4,8,16,8 #16,32,64,32 # 8,16,32,16
    model = Sequential([
        
        AveragePooling2D(input_shape=single_input_shape),

        Conv2D(lay1, kernel_size=(3,3), strides=(1, 1),\
                   kernel_regularizer=l2(l2_reg), kernel_initializer=initializer),
        ReLU(),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        Dropout(0.2),

        Conv2D(lay2, kernel_size=(3,3), strides=(1, 1), data_format = 'channels_last',\
                   kernel_regularizer=l2(l2_reg), kernel_initializer=initializer),
        ReLU(),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        
        Flatten(),
        Dropout(0.2),

        Conv2D(lay3, kernel_size=(3,3), strides=(2, 2), data_format = 'channels_last',\
                   kernel_regularizer=l2(l2_reg), kernel_initializer=initializer),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),

        Flatten(),
        Dropout(0.2),

        Dense(dense, activation='relu'),
        BatchNormalization(),
    #     GlobalAveragePooling2D(),
        Dense(1, activation='sigmoid', dtype=tf.float32, kernel_initializer=initializer)
    ])

    return model

def relu_max(x):
    return relu(x, max_value=1)

def single_functional_feedforward_model():
    l2_reg = 1e-05
    gn = 5e-04
    initializer = tf.keras.initializers.he_normal(seed=1) #TruncatedNormal(mean=0.0, stddev=0.07)
    img_input = Input(shape=single_input_shape)
#     x = GaussianNoise(gn)(img_input)
#     x = Reshape((single_input_shape))(x)
    lay1,lay2,lay3,dense = 8,16,32,8 #32,64,128,32  16,32,64,16
    x = Conv2D(lay1, kernel_size=(3,3), input_shape=single_input_shape, strides=(1, 1), data_format = 'channels_last',\
               kernel_regularizer=l2(l2_reg), kernel_initializer=initializer)(img_input)
    x = ReLU()(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
#     x = Dropout(0.1)(x)
    x = Conv2D(lay2, kernel_size=(3,3), strides=(1, 1), kernel_regularizer=l2(l2_reg),\
               kernel_initializer=initializer)(x)
    x = ReLU()(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
#     x = Dropout(0.1)(x)
    x = Conv2D(lay3, kernel_size=(3,3), strides=(1, 1), kernel_regularizer=l2(l2_reg),\
               kernel_initializer=initializer)(x)
    x = ReLU()(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
    x = Flatten()(x)
#     x = Dropout(0.1)(x)
    x = Dense(dense, kernel_initializer=initializer)(x)
    x = ReLU()(x)
    x = BatchNormalization()(x)
    # x = Dense(1, activation=relu_max, dtype=tensorflow.float32)(x)
#     x = Flatten()(x)
    out_metric = Dense(1, activation='sigmoid', dtype=tf.float32, name=metric, kernel_initializer=initializer)(x)
    model = Model(inputs=img_input, outputs=out_metric)
    
    return model

def joe_test_feedforward_model(): # training pearson = 0.75 vaidation = 0.11
    l2_reg = 0 #0.0005
    gn = 0.005
    initializer = tf.keras.initializers.he_normal(seed=1)
    #initializer = tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.07)
    img_input = Input(shape=single_input_shape)
    x = ZeroPadding2D((3, 3))(img_input)
    lay1,lay2,lay3,dense = 16,32,64,16 #16,32,64,16
    x = Conv2D(lay1, kernel_size=(3,3), input_shape=single_input_shape, strides=(2,2), data_format = 'channels_last',\
               kernel_regularizer=l2(l2_reg), padding = 'same', kernel_initializer=initializer)(img_input)
    x = BatchNormalization()(x)
#     x = Dropout(0.1)(x)
    x = ReLU()(x)
    x_skip = x
    x = Conv2D(lay1, kernel_size=(3,3), input_shape=single_input_shape, strides=2, data_format = 'channels_last',\
               kernel_regularizer=l2(l2_reg), padding = 'same', kernel_initializer=initializer)(img_input)
    x = BatchNormalization()(x)
    x = Add()([x, x_skip]) 
    print(x.shape)
    x = ReLU()(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
#     x = Dropout(0.1)(x)
    x_skip = x
    x = Conv2D(lay2, kernel_size=(3,3), strides=2, kernel_regularizer=l2(l2_reg), padding = 'same', kernel_initializer=initializer)(x)
    x = BatchNormalization()(x)
    # x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
    x = ReLU()(x)
    x = Conv2D(lay2, kernel_size=(3,3), strides=(1, 1), kernel_regularizer=l2(l2_reg),  padding = 'same', kernel_initializer=initializer)(x)
    x = BatchNormalization()(x)
    x_skip = Conv2D(32, (1,1), strides = (2,2), kernel_initializer=initializer)(x_skip)
    # Add Residue
    x = Add()([x, x_skip])  
    x = ReLU()(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
    x = Conv2D(lay3, kernel_size=(3,3), strides=(1, 1), kernel_regularizer=l2(l2_reg), kernel_initializer=initializer)(x)
    x = ReLU()(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(x)
    x = Flatten()(x)
    x = Dropout(0.2)(x)
    x = Dense(dense, kernel_initializer=initializer)(x)
    x = ReLU()(x)
#     x = ELU(0.3)(x)
    x = BatchNormalization()(x)
    # x = Dense(1, activation=relu_max, dtype=tensorflow.float32)(x)
#     x = Flatten()(x)
    out_metric = Dense(1, activation='sigmoid', dtype=tf.float32, name=metric, kernel_initializer=initializer)(x)
    model = Model(inputs=img_input, outputs=out_metric)
    
    return model

### Other Models

In [ ]:
# AUTOENCODER MODEL
def build_autoencoder(input_shape):
    l2_reg = 0
    gn = 0 #0.0005
#     initializer = tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.07) #he_normal(seed=1)
#     img_input = Input(shape=single_input_shape)
#     GaussianNoise(gn)(img_input)
#     Reshape((single_input_shape))
    lay1,lay2,lay3,dense = 16,8,8 ,64 #4,8,16,8 #16,32,64,32 # 8,16,32,16
    
    input_img = Input(shape=input_shape)
    
    x = Conv2D(lay1, (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(lay2, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(lay3, (3, 3), activation='relu', padding='same')(x)
    encoder = MaxPooling2D((2, 2), padding='same')(x)

    # at this point the representation is (32, 32, 8)

    x = Conv2D(lay3, (3, 3), activation='relu', padding='same')(encoder)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(lay2, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(lay1, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    decoder = Conv2D(1, (3, 3), activation=relu_max, padding='same')(x)
    
    autoencoder = Model(input_img, decoder)

    return encoder, decoder, autoencoder

## Compile the Model

### Introduction and Compiling

The `compile` method configures the learning process. This involves specifying:
* An optimizer https://keras.io/optimizers/. This is what controls how the model updates based on the value of the loss function.
* A loss fucntion https://keras.io/losses/
* A list of metrics 

**Defining the shape values as variables**

In [ ]:
x_im, y_im, dvf_contour_im = 256,256,2 # 256,256,4 # 384,384,3   # 512,512,3 # 256,256,3
single_input_shape = (x_im, y_im, dvf_contour_im)
flat_single_input_shape = (x_im*y_im*dvf_contour_im)

image_input = (256, 256, 1)

**Assign model as a variable and compile**

In [ ]:
from tensorflow.keras import optimizers 
from tensorflow.keras import metrics
#from tensorflow.keras.optimizers import Adam, RMSprop
# Grab the model from our avaliable set of models

# model = single_functional_feedforward_model()

model = joe_test_feedforward_model()
# model = vector_model()

# model = vector_model_sequential()

# model = vector_avg_pooling_model()

# model = vector_feedforward_model()

#Compile model - we tried two different optimizers, one with momentum (adam) and one without
#Adadelta = tensorflow.optimizers.Adadelta() #Default values: learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False

Adam = tf.optimizers.Adam(learning_rate=1e-03)
RMSprop = tf.optimizers.RMSprop(lr=1e-03) #SGD #lr=float(learning_rate), momentum=0.9, nesterov=True
# Adadelta = tf.optimizers.Adadelta(lr=1.5, rho=0.95, epsilon=1e-08, decay=0.001)
optimizer_choice_string = "Adam" # "Adam" "RMSprop" "Adadelta"
optimizer_choice = Adam #RMSprop #Adam #Adadelta
loss_fn = "mean_squared_error" # sparse_categorical_crossentropy mean_squared_error
model.compile(loss=loss_fn, optimizer=optimizer_choice)

model.summary()

In [ ]:
# If using an autoencoder for the image input. Not currently used as a part of the model but will be left here in case of future interest
autoencoder = build_autoencoder(image_input)[2]
ae_loss_fn = 'mean_squared_error'
autoencoder.compile(loss=ae_loss_fn, optimizer=optimizer_choice)

autoencoder.summary()

## Train the Model

### Preparing the Training and Validation datasets

The `fit` function is used for training. An epoch is 'one complete presentation of the data set to be learned to a learning machine'. Many epochs are needed to train a neural network.

If loading in the data use the following code

In [ ]:
# Normalise any array (e.g. pixel brightness) for chosen upper and lower bounds and indexes
def normalise_multiple(array, lower_bound, upper_bound, lower_index, upper_index):
            
    if lower_index==0:
        component='pixel brightness'
    if lower_index==1 and upper_index==2:
        component='DVF x'    
    if lower_index==1 and upper_index==3:
        component='DVF x & y'
    if lower_index==2 and upper_index==3:
        component='DVF y'
    
    print("Normalising {} component of slices...".format(component))
    
    norm_array = array[:,:,:,lower_index:upper_index] if array.ndim==4 else array

    max_value = np.max(norm_array); min_value = np.min(norm_array)
    norm_array = ((upper_bound-lower_bound)*( (norm_array-min_value) / (max_value-min_value) )) + lower_bound

    print("Min value: ", np.min(norm_array), "\t Max value: ", np.max(norm_array))

    return norm_array

# Normalise two separate arrays with respect to their joint array
def normalise_train_and_validate(array1, array2, lower_bound, upper_bound, lower_index, upper_index):
#     This is the formatting of the data i.e. first dimension is pixel brightness, then DVF_x,y...
    if lower_index==0 and upper_index==1:
        component='pixel brightness'
    if lower_index==1 and upper_index==2:
        component='DVF x'    
    if lower_index==1 and upper_index==3:
        component='DVF x & y'
    if lower_index==2 and upper_index==3:
        component='DVF y'
    if lower_index==3:
        component='reference contour'
        
    print("Normalising {} component of slices...".format(component))
    
    if array1.ndim==4 and array2.ndim==4:
        norm_array1 = array1[:,:,:,lower_index:upper_index]
        norm_array2 = array2[:,:,:,lower_index:upper_index]
        
        min_value = np.min(norm_array1) if np.min(norm_array1) <= np.min(norm_array2) else np.min(norm_array2)
        max_value = np.max(norm_array1) if np.max(norm_array1) >= np.max(norm_array2) else np.max(norm_array2)
        
#         print("Min value: ", min(np.min(norm_array1), np.min(norm_array2)),\
#       "\t Max value: ", max(np.max(norm_array1), np.max(norm_array2)))
        
    else:
        norm_array1 = array1
        norm_array2 = array2
#         min and max of the joint array
        min_value = np.min(norm_array1) if np.min(norm_array1) <= np.min(norm_array2) else np.min(norm_array2)
        max_value = np.max(norm_array1) if np.max(norm_array1) >= np.max(norm_array2) else np.max(norm_array2)
        
#         print("Min value: ", min(np.min(norm_array1), np.min(norm_array2)),\
#       "\t Max value: ", max(np.max(norm_array1), np.max(norm_array2)))
    # this is the equation for normalising any array
    norm_array1 = ((upper_bound-lower_bound)*( (norm_array1-min_value) / (max_value-min_value) )) + lower_bound
    norm_array2 = ((upper_bound-lower_bound)*( (norm_array2-min_value) / (max_value-min_value) )) + lower_bound

    print("Min value: ", min(np.min(norm_array1), np.min(norm_array2)),\
          "\t Max value: ", max(np.max(norm_array1), np.max(norm_array2)))
    
    if array1.ndim==4 and array2.ndim==4:
        if upper_index-lower_index==1:
            norm_array1 = np.reshape(norm_array1, (len(norm_array1), x_im, y_im))
            norm_array2 = np.reshape(norm_array2, (len(norm_array2), x_im, y_im)) 
        else:
            norm_array1 = np.reshape(norm_array1, (len(norm_array1), x_im, y_im, upper_index-lower_index))
            norm_array2 = np.reshape(norm_array2, (len(norm_array2), x_im, y_im, upper_index-lower_index))
    # arrays must be reshaped otherwise an error will occur
    return norm_array1, norm_array2

# Identify slices where the DVFs are constant i.e. invalid for use, or remove DVFs above a certain value
def remove_invalid_DVF_slices(array, lower_index, upper_index, min_dvf, max_dvf, dvf_cut_off=None):
    print('Removing slices containing invalid DVFs...')
    invalid_slice, cut_off = [], []
    for i in range(len(array)):
        if i%1000==0:
            print(i)
        if upper_index-lower_index==2: #and lower_index!=0:
            DVF_array1 = array[i,:,:,lower_index]
            DVF_array2 = array[i,:,:,upper_index-1]
            
            if dvf_cut_off==True: # bool choice to remove DVF above or below certain value
                if max_dvf<np.max(DVF_array1) or max_dvf<np.max(DVF_array2):
                    invalid_slice.append(i)
                if min_dvf>np.min(DVF_array1) or min_dvf>np.min(DVF_array2):
                    invalid_slice.append(i)
                    
            elif np.min(DVF_array1)==np.max(DVF_array1) or np.min(DVF_array2)==np.max(DVF_array2):
#                 print(np.min(DVF_array1), np.max(DVF_array1), np.min(DVF_array2), np.max(DVF_array2))
                invalid_slice.append(i)
            elif np.min(DVF_array1)==np.max(DVF_array1) and np.min(DVF_array2)==np.max(DVF_array2):
                invalid_slice.append(i)
            else:
                pass
                
    invalid_slices = np.array(invalid_slice)
#     cut_off_slices = np.array(cut_off)
#     valid_cut_off_slices = np.argwhere(np.isin(np.arange(len(array)), cut_off_slices, invert=True)).ravel()
    valid_slices = np.argwhere(np.isin(np.arange(len(array)), invalid_slices, invert=True)).ravel()
    
    return array[valid_slices], valid_slices, invalid_slices  #, array[valid_cut_off_slices], cut_off_slices

# Remove DVF values where the pixel brightness is zero - didn't improve results when we tried but may still be useful
def remove_zero_pixel_DVFs(array, lower_index, upper_index):
    print('Removing DVF values where the prixel brightness is 0 ...')
    zero_pixel_indexes = np.argwhere(array[:,:,:,0]==0)
    for i in range(len(zero_pixel_indexes)):
#         if zero_pixel_indexes[i,0]%250==0 and zero_pixel_indexes[i,1]%250==0 and zero_pixel_indexes[i,2]%250==0:
#             print(zero_pixel_indexes[i,0])
        array[zero_pixel_indexes[i,0], zero_pixel_indexes[i,1], zero_pixel_indexes[i,2], lower_index:upper_index] = 0
        
    return array

In [ ]:
# if organ=='BRAIN_STEM':
#     n_train_max = 
#     n_validate_max = 
# Choose to include flipped images or not
flipped = False

if organ=='PAROTID_LT':
    if flipped==True:
        n_train_max = 28706
        n_validate_max = 18652
    else:
        n_train_max = 14353
        n_validate_max = 9326
    
# Choose number of training and testing samples to use
n_train = n_train_max  #n_train_max 
n_validate = n_validate_max  #n_validate_max 

# Choose data type of arrays
data_type = 'float16'

# Choose either to remove zero dice scores, train CNN or autoencoder
remove_zero_dice_scores = False
train_CNN = True
train_AE = False

# Set parameters for normalising either the pixel brightness or DVF components
Image_norm_min = 0; Image_norm_max = 1
DVF_norm_min = -1; DVF_norm_max = 1

# # slices of shape (n, x, y, [dvf_x, dvf_y])

# Set dvf removal parameters
bool_cut_slices = False; dvf_min_cut_off = -3500; dvf_max_cut_off = 3500

In [ ]:
# Folders for accessing and loading the slice data
organ = 'PAROTID_LT'
slice_array_folder = '/hepgpu9-data1/sbridger/mphys_2d_network_2021-22/'\
'slice_arrays/new_registrations/{}/complete_arrays/'.format(organ)
train_folder = slice_array_folder+'train_odd_{}_'.format(organ)
test_folder = slice_array_folder+'test_even_{}_'.format(organ)

# Set as this if normalising and transforming metric arrays after removing invalid DVFs etc.
metric_choice = 'dice'
train_metric_array = np.load(train_folder+'multiple_{}_array.npy'.format(metric_choice), mmap_mode='c')[:n_train]
test_metric_array = np.load(test_folder+'multiple_{}_array.npy'.format(metric_choice), mmap_mode='c')[:n_validate]
Metrics = np.concatenate((train_metric_array, test_metric_array)) 

if remove_zero_dice_scores and metric_choice=='dice':
    print('Removing zero dice scores...')
    nz_dice_scores_train = np.where(Metrics[:n_train]>0)[0]
    nz_dice_scores_validate = np.where(Metrics[n_train:]>0)[0]
    
    train_indices = nz_dice_scores_train[:n_train]
    validate_indices = nz_dice_scores_validate[:n_validate]
    
#     X_train_mmap = np.load('PAROTID_LT_multiple_train_patient_image_dvf_xy_256_array.npy', mmap_mode = 'c')[train_indices]#.astype(data_type)
#     X_validate_mmap = np.load('PAROTID_LT_multiple_validate_patient_image_dvf_xy_256_array.npy', mmap_mode = 'c')[validate_indices]#.astype(data_type)
    
    X_train_mmap = np.load('PAROTID_LT_multiple_train_ref_image_reg_contour_{}_array.npy'.format(x_im), mmap_mode = 'c')[train_indices]
    X_validate_mmap = np.load('PAROTID_LT_multiple_validate_ref_image_reg_contour_{}_array.npy'.format(x_im), mmap_mode = 'c')[validate_indices]
    
#     X_train_mmap = np.load('PAROTID_LT_multiple_train_ref_image_reg_contour_plus_LR_flipped_{}_normalised_array.npy'.format(x_im), mmap_mode = 'c')[:n_train]
#     X_validate_mmap = np.load('PAROTID_LT_multiple_validate_ref_image_reg_contour_plus_LR_flipped_{}_normalised_array.npy'.format(x_im), mmap_mode = 'c')[:n_validate]

    print('Initial shapes of training and validation data:')
    print(X_train_mmap.shape, X_validate_mmap.shape)
    
#     Remove slices with invalid DVFs
#     X_train_mmap, valid_train_indexes =\
#     remove_invalid_DVF_slices(X_train_mmap, 1, 3, dvf_min_cut_off, dvf_max_cut_off, bool_cut_slices)
    
#     X_validate_mmap, valid_validation_indexes =\
#     remove_invalid_DVF_slices(X_validate_mmap, 1, 3, dvf_min_cut_off, dvf_max_cut_off, bool_cut_slices)
    
#     print('Shapes of training and validation data after removing slices with invalid DVFs:')
#     print(X_train_mmap.shape, X_validate_mmap.shape)
    
#     Normalise images

    X_train_mmap[:,:,:,0], X_validate_mmap[:,:,:,0] =\
    normalise_train_and_validate(X_train_mmap, X_validate_mmap, Image_norm_min, Image_norm_max, 0, 1)
    
#     Normalise DVF x & y components
#     X_train_mmap[:,:,:,1:3], X_validate_mmap[:,:,:,1:3] =\
#     normalise_train_and_validate(X_train_mmap, X_validate_mmap, DVF_norm_min, DVF_norm_max, 1, 3)


if train_CNN:
    print('Preparing CNN slice data...')
    
#     X_train_mmap = np.load('PAROTID_LT_multiple_train_patient_image_dvf_xy_256_array.npy', mmap_mode = 'c')[:n_train]
#     X_validate_mmap = np.load('PAROTID_LT_multiple_validate_patient_image_dvf_xy_256_array.npy', mmap_mode = 'c')[:n_validate]
    
#     X_train_mmap = np.load('PAROTID_LT_multiple_train_ref_image_reg_contour_{}_array.npy'.format(x_im), mmap_mode = 'c')[:n_train]
#     X_validate_mmap = np.load('PAROTID_LT_multiple_validate_ref_image_reg_contour_{}_array.npy'.format(x_im), mmap_mode = 'c')[:n_validate]
    
#     X_train_mmap = np.load('PAROTID_LT_multiple_train_ref_image_dvf_xy_reg_contour_{}_array.npy'.format(x_im), mmap_mode = 'c')[:n_train]
#     X_validate_mmap = np.load('PAROTID_LT_multiple_validate_ref_image_dvf_xy_reg_contour_{}_array.npy'.format(x_im), mmap_mode = 'c')[:n_validate]    


    X_train_mmap = np.load('PAROTID_LT_multiple_train_ref_image_reg_contour_plus_LR_flipped_{}_normalised_array.npy'.format(x_im), mmap_mode = 'c')[:n_train]
    X_validate_mmap = np.load('PAROTID_LT_multiple_validate_ref_image_reg_contour_plus_LR_flipped_{}_normalised_array.npy'.format(x_im), mmap_mode = 'c')[:n_validate]
    
    print('Initial shapes of training and validation data:')
    print(X_train_mmap.shape, X_validate_mmap.shape)
    
#     Remove slices with invalid DVFs
#     X_train_mmap, valid_train_indexes, invalid_train_indexes =\
#     remove_invalid_DVF_slices(X_train_mmap, 1, 3, dvf_min_cut_off, dvf_max_cut_off, bool_cut_slices)
    
#     X_validate_mmap, valid_validation_indexes, invalid_validation_indexes =\
#     remove_invalid_DVF_slices(X_validate_mmap, 1, 3, dvf_min_cut_off, dvf_max_cut_off, bool_cut_slices)
    
#     print('Shapes of training and validation data after removing slices with invalid DVFs:')
#     print(X_train_mmap.shape, X_validate_mmap.shape)

#     Normalise images & contour images:

#     Take log(1+PB) of pixel brightness

#     X_train_mmap[:,:,:,0], X_validate_mmap[:,:,:,0] =\
#     np.log(X_train_mmap[:,:,:,0]+1), np.log(X_validate_mmap[:,:,:,0]+1)

#     X_train_mmap[:,:,:,0], X_validate_mmap[:,:,:,0] =\
#     normalise_train_and_validate(X_train_mmap, X_validate_mmap, Image_norm_min, Image_norm_max, 0, 1)

#     X_train_mmap[:,:,:,3], X_validate_mmap[:,:,:,3] =\
#     normalise_train_and_validate(X_train_mmap, X_validate_mmap, Image_norm_min, Image_norm_max, 3, 4)
    
#     Remove DVFs where the pixel brightness is 0:
#     X_train_mmap = remove_zero_pixel_DVFs(X_train_mmap, 1, 3)
#     X_validate_mmap = remove_zero_pixel_DVFs(X_validate_mmap, 1, 3)

#     Normalise DVF x & y components    
#     X_train_mmap[:,:,:,1:3], X_validate_mmap[:,:,:,1:3] =\
#     normalise_train_and_validate(X_train_mmap, X_validate_mmap, DVF_norm_min, DVF_norm_max, 1, 3)

if train_AE:
    print('Preparing autoencoder slice data...')
    X_train_mmap_load = np.load('PAROTID_LT_multiple_train_patient_image_dvf_xy_256_array.npy', mmap_mode = 'c')[:n_train,:,:,0]#.astype(data_type)
    X_validate_mmap_load = np.load('PAROTID_LT_multiple_validate_patient_image_dvf_xy_256_array.npy', mmap_mode = 'c')[:n_validate,:,:,0]#.astype(data_type)
#     print(X_train_mmap.shape, X_validate_mmap.shape)

#     Remove slices with invalid DVFs
#     X_train_mmap_load, valid_train_indexes =\
#     remove_invalid_DVF_slices(X_train_mmap, 1, 3, dvf_min_cut_off, dvf_max_cut_off, bool_cut_slices)
    
#     X_validate_mmap_load, valid_validation_indexes =\
#     remove_invalid_DVF_slices(X_validate_mmap, 1, 3, dvf_min_cut_off, dvf_max_cut_off, bool_cut_slices)
    
    #     Normalise images
    X_train_mmap_norm, X_validate_mmap_norm =\
    normalise_train_and_validate(X_train_mmap_load, X_validate_mmap_load, Image_norm_min, Image_norm_max, 0, 1)
    
    X_train_mmap = np.reshape(X_train_mmap_norm, (len(X_train_mmap_norm), x_im, y_im, 1))
    X_validate_mmap = np.reshape(X_validate_mmap_norm, (len(X_validate_mmap_norm), x_im, y_im, 1))
    
    print(X_train_mmap.shape, X_validate_mmap.shape)


In [ ]:
# If you want to include data where the images have been flipped then uncomment this code
# X_train_mmap_flipped = np.empty(X_train_mmap.shape)
# X_validate_mmap_flipped = np.empty(X_validate_mmap.shape)

# for i in range(len(X_train_mmap)):
#     for j in range(2):
#         X_train_mmap_flipped[i,:,:,j] = np.fliplr(X_train_mmap[i,:,:,j])
    
# for i in range(len(X_validate_mmap)): 
#     for j in range(2):
#         X_validate_mmap_flipped[i,:,:,j] = np.fliplr(X_validate_mmap[i,:,:,j])

# Just a test to see the effect of the tranformed images
# for i in range(1):
#     for j in range(2):
#         print(i,j)
#         plt.imshow(X_train_mmap[i,:,:,j].astype(np.float32))
#         plt.show()
#         plt.imshow(X_train_mmap_flipped[i,:,:,j].astype(np.float32))
#         plt.show()
#         plt.imshow(X_validate_mmap[i,:,:,j].astype(np.float32))
#         plt.show()
#         plt.imshow(X_validate_mmap_flipped[i,:,:,j].astype(np.float32))
#         plt.show()

In [ ]:
#Pixel brightness/ DVF histogram for visualisation of the data
# pixel_array = np.concatenate(( X_train_mmap[:,:,:,0], X_validate_mmap[:,:,:,0] )).astype(np.float32)
# array = pixel_array.ravel()
# n_bins = 50
# fig, ax1 = plt.subplots(figsize=(5, 3.5))
# #removes the grey background to the plot
# fig.patch.set_facecolor('white')
# plt.hist(array, weights=None, density = False, bins = n_bins)
# plt.title('Log pixel brightness distribution', y=1.12)
# plt.xlabel('Log pixel brightness')
# plt.ylabel('Frequency')

# # plt.title('DVF y-component distribution', y=1.12)
# # plt.xlabel('DVF y-component value')
# plt.ylim(0,3.5e8)
# # plt.xlim(0,1)
# plt.tick_params(axis='both', which='major')
# plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0), useMathText=True)
# plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0), useMathText=True)
# plt.tight_layout()
# plt.show()

# for i in range(1000):#len(pixel_array)):
# #     pixel_array = X_train_mmap_test[i,:,:,0]
# #     pixel_array32 = np.float32(pixel_array)
#     #print(pixel_array32.dtype)
#     im_array = pixel_array[i,:,:]
#     fig = plt.figure(figsize=(5, 3.5))
#     fig.patch.set_facecolor('white')
#     ax = fig.add_subplot(111)
#     ax.set_title('Pixel brightness colour map')
#     plt.imshow(im_array)
#     ax.set_aspect('equal')

#     #cax = fig.add_axes([0.12, 0.1, 0.78, 0.8])
#     # cax.get_xaxis().set_visible(False)
#     # cax.get_yaxis().set_visible(False)
#     # cax.patch.set_alpha(0)
#     # cax.set_frame_on(False)
#     plt.colorbar(orientation='vertical')
# #     plt.clim(0, 4700)
#     plt.show()

In [ ]:
# Saving image arrays by joining them up in the manner you see fit - again, adapt this to save time if creating your
# own data sets.
# x1 = np.reshape(X_train_mmap[:,:,:,0], (len(X_train_mmap),256,256,1)) 
# x2 = np.reshape(X_train_mmap[:,:,:,3], (len(X_train_mmap),256,256,1))
# print(x1.shape, x2.shape)
# x3 = np.reshape(X_validate_mmap[:,:,:,0], (len(X_validate_mmap),256,256,1)) 
# x4 = np.reshape(X_validate_mmap[:,:,:,3], (len(X_validate_mmap),256,256,1))
# X_train_mmap = np.concatenate((x1,x2), axis=3) # X_train_mmap
# print(X_train_mmap.shape)
# X_validate_mmap = np.concatenate((x3,x4), axis=3) # X_validate_mmap
# X_train_mmap = X_train_mmap[:,:,:,0]
# X_validate_mmap = X_validate_mmap[:,:,:,0]
# X_train_mmap = np.reshape(X_train_mmap, (len(X_train_mmap),x_im,x_im,1))
# X_validate_mmap = np.reshape(X_validate_mmap, (len(X_validate_mmap),x_im,x_im,1))

# X_train_def_contours = np.load(slice_array_folder+'train_odd_PAROTID_LT_multiple_def_contour_image_array.npy', mmap_mode='c')[:,128:-128,128:-128]
# X_validate_def_contours = np.load(slice_array_folder+'test_even_PAROTID_LT_multiple_def_contour_image_array.npy', mmap_mode='c')[:,128:-128,128:-128]
# print('here')
# X_train_def_contours = np.reshape(X_train_def_contours, (len(X_train_def_contours),x_im,x_im,1))
# X_validate_def_contours = np.reshape(X_validate_def_contours, (len(X_validate_def_contours),x_im,x_im,1))
# print(X_train_def_contours.shape, X_train_mmap.shape)
# X_train_joined = np.concatenate((X_train_mmap, X_train_def_contours), axis=3)
# X_validate_joined = np.concatenate((X_validate_mmap, X_validate_def_contours), axis=3)

# np.save('PAROTID_LT_multiple_train_image_ref_reg_contour_{}_array.npy'.format(x_im), X_train_joined)
# np.save('PAROTID_LT_multiple_validate_image_ref_reg_contour_{}_array.npy'.format(x_im), X_validate_joined)

# np.save('PAROTID_LT_multiple_train_ref_reg_contour_image_dvf_array_{}_array.npy'.format(x_im), X_train_joined)
# np.save('PAROTID_LT_multiple_validate_ref_reg_contour_image_dvf_array_{}_array.npy'.format(x_im), X_validate_joined)

# np.save('PAROTID_LT_multiple_train_ref_image_dvf_reg_contour_{}_array.npy'.format(x_im), X_train_joined)
# np.save('PAROTID_LT_multiple_validate_ref_image_dvf_reg_contour_{}_array.npy'.format(x_im), X_validate_joined)
# print(X_train_joined.shape)

# For downsizing the images change the value of cut:
#     cut = 128
#     X_train_mmap = np.load(train_folder+'multiple_ref_contour_image_array.npy', mmap_mode = 'c')[:,cut:-cut,cut:-cut]
#     X_validate_mmap = np.load(test_folder+'multiple_ref_contour_image_array.npy', mmap_mode = 'c')[:,cut:-cut,cut:-cut]
#     np.save('PAROTID_LT_multiple_train_ref_contour_image_array_256_array.npy', X_train_mmap)
#     np.save('PAROTID_LT_multiple_validate_ref_contour_image_array_256_array.npy', X_validate_mmap)

In [ ]:
# #     Metrics & Weights
metric_choice = 'dice'
train_metric_array = np.load(train_folder+'multiple_{}_array.npy'.format(metric_choice), mmap_mode='c')[:n_train]
test_metric_array = np.load(test_folder+'multiple_{}_array.npy'.format(metric_choice), mmap_mode='c')[:n_validate]
Metrics = np.concatenate((train_metric_array, test_metric_array)) 

valid_train_indexes = np.arange(len(train_metric_array))
valid_validation_indexes = np.arange(len(train_metric_array),len(train_metric_array)+len(test_metric_array))
valid_indexes = np.concatenate((valid_train_indexes, valid_validation_indexes))

if metric_choice == "rms" or metric_choice == "msd":
    power = 1
else:
    power = 1

Metrics = transformed_array(Metrics[valid_indexes], power, metric_choice)


if flipped==True:
    Y_train_copy = Metrics[valid_train_indexes].astype(data_type)
    Y_validate_copy = Metrics[valid_validation_indexes].astype(data_type)

    Y_train_mmap = np.concatenate((Y_train_copy, Y_train_copy))
    Y_validate_mmap = np.concatenate((Y_validate_copy, Y_validate_copy))
    
elif remove_zero_dice_scores and metric_choice=='dice':
    print('Zero dice scores metric array chosen')
    Y_train_mmap = Metrics[train_indices].astype(data_type)
    Y_validate_mmap = Metrics[validate_indices].astype(data_type)
    
else:
    Y_train_mmap = Metrics[valid_train_indexes].astype(data_type)
    Y_validate_mmap = Metrics[valid_validation_indexes].astype(data_type)

# Weights
weight_array = binningWeights(Metrics, n_bins, 3)# weightArray
weight_array, mn, mx = normalise(weight_array)
W_train_mmap = weight_array[valid_train_indexes].astype(data_type)

# W_train_mmap = weight_array[train_indices].astype(data_type)

In [ ]:
label_list = ["training ", "validation "]
# n_bins=60
if train_CNN or remove_zero_dice_scores:
    for i in range(2):
        fig, ax1 = plt.subplots(figsize=(5, 3.5))
        Y_array=[Y_train_mmap, Y_validate_mmap]
        #removes the grey background to the plot
        fig.patch.set_facecolor('white')
        plt.hist(Y_array[i], weights=None, density = False, bins = n_bins, range = (0,1))
        ax1.set_ylabel('Frequency', fontsize=18)
        ax1.set_xlabel(label_list[i] + metric_choice, fontsize=18)
        plt.tick_params(axis='both', which='major', labelsize=14)
        plt.tight_layout()
        plt.show()

### Training

This built-in tensorflow function **Trains** the model for a fixed number of epochs(iterations on a dataset)
```
model.fit(
    x=None, y=None, batch_size=None, epochs=1, verbose='auto',
    callbacks=None, validation_split=0.0, validation_data=None, shuffle=True,
    class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None,
    validation_steps=None, validation_batch_size=None, validation_freq=1,
    max_queue_size=10, workers=1, use_multiprocessing=False
)
```
* https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit - details what all the input parameters mean
* https://medium.com/analytics-vidhya/train-keras-model-with-large-dataset-batch-training-6b3099fdf366
* https://medium.com/analytics-vidhya/write-your-own-custom-data-generator-for-tensorflow-keras-1252b64e41c3

In [ ]:
# If using autoencoder
# batch_size = 32
# epoch_choice = 4

# x_train = X_train_mmap[:,:,:,0] # X_train_mmap
# x_validate = X_validate_mmap[:,:,:,0] # X_validate_mmap
# x_train = np.reshape(x_train, (len(x_train), x_im, y_im, 1))
# x_validate = np.reshape(x_validate, (len(x_validate), x_im, y_im, 1))
# # Train autoencoder

# history = autoencoder.fit(x=x_train, y=x_train, batch_size=batch_size, shuffle=True,
#                     validation_data = (x_validate, x_validate),
#           epochs=epoch_choice, verbose=1)#, callbacks=callback_list) #[cp_callback])

# # Construct the images
# # X_train = X_train_mmap; X_validate = X_validate_mmap
# X_train = x_train
# X_validate = x_validate

# AE_predicted_metric_training = autoencoder.predict(X_train, batch_size=batch_size, verbose=0)
# AE_predicted_metric_validation = autoencoder.predict(X_validate, batch_size = batch_size, verbose=0)

# AE_predicted_training = np.reshape(AE_predicted_metric_training, (len(X_train),256,256))
# AE_predicted_validation = np.reshape(AE_predicted_metric_validation, (len(X_validate),256,256))

# # View the reconstructed validation images
# n_view = 3
# for i in range(n_view):
#     f, axarr = plt.subplots(1,2)
# #     axarr[0].imshow(X_validate[i,:,:,0])
#     axarr[1].imshow(AE_predicted_validation[i])
#     plt.show()

In [ ]:
from tensorflow.keras import callbacks

learning_rate = '0.005'
momentum = 0.9
patience = 20

output_folder = '/hepgpu9-data1/sbridger/mphys_2d_network_2021-22/plots/' + organ + '/'

ES = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=int(patience), verbose=1, mode='min', baseline=None, \
                            restore_best_weights=True)

class LearningRateReducer(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs={}):
        old_lr = self.model.optimizer.lr.read_value()
        new_lr = old_lr * 0.99
        print("\nEpoch: {}. Reducing Learning Rate from {} to {}".format(epoch, old_lr, new_lr))
        self.model.optimizer.lr.assign(new_lr)

reduce_lr_callbacks = [LearningRateReducer()]
#callback_list = [ES, TB]
#rlrop = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10)

callback_list = [ES]

# Checkpoint callback, period = x means save the model weights after every x epochs.
# {epoch:04d} refers to the string of the epoch number, i.e. naming the files by epoch number
checkpoint_path='/hepgpu9-data1/sbridger/mphys_2d_network_2021-22/weight_checkpoints/final_model/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)
# cp_name = 
cp_callback = callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    save_freq=1, 
    save_weights_only=True,
    verbose=1)

In [ ]:
x_train = X_train_mmap
x_validate = X_validate_mmap
y_train = Y_train_mmap # Y_train_mmap
y_validate = Y_validate_mmap # Y_validate_mmap
# w_train = W_train_mmap
# w_train /= np.mean(w_train)
# print(np.mean(w_train), np.std(w_train))
# w_train = binningWeights(Metrics, n_bins, 0)

# Allow for autoencoded output as CNN input
# x_train[:,:,:,0] = AE_predicted_training
# x_validate[:,:,:,0] = AE_predicted_validation

# SPECIFYING IMPORTANT FUNCTION PARAMETERS
batch_size = 32
epoch_choice = 80

# Train CNN

history = model.fit(x=x_train, y=y_train, batch_size=batch_size,
                    validation_data = (x_validate, y_validate),# sample_weight = w_train,
                    shuffle=True, epochs=epoch_choice, verbose=1, callbacks=[cp_callback])

`ImageDataGenerator` is a Python class with many objects that allow you to manipulate the images, such as horizontal flip, vertical flip, rotation. It converts images into tensor image data. Using the above objects you can apply transformations to your training set, which may allow the model to generalise better.
* If our data is in a directory we use `flow_from_directory`, this is an attribute of our `ImageDataGenerator` class. 
* If our data is in a dataframe we use `flow_from_dataframe`, this is an attribute of our `ImageDataGenerator` class.
* This class can also rescale all pixel values between 0 and 1 `train_datagen = ImageDataGenerator(rescale=1./255)`, for training, `valid_datagen = ImageDataGenerator(rescale=1./255)`, for validation.
* We use the `datagen.flow` built-in function when the data is stored as a variable accessible in the code. 

```
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator()
start0 = time.time()
train_generator = datagen.flow(X_train, [Y_train1, Y_train2], batch_size=batch_size, shuffle=True,
                               sample_weight={metric1: W_train1, metric2: W_train2}, seed=1)

validation_generator = datagen.flow(X_validate, [Y_validate1, Y_validate2], 
                                    batch_size=batch_size, shuffle=True, seed=1,
                                    sample_weight={metric1: W_validate1, metric2: W_validate2})
stop0 = time.time()
print("It took ", str(stop0-start0), " seconds to load in the data.")
```

**Callbacks**

`tensorflow.keras.callbacks` has many attributes two of which are: 

* `LearningRateScheduler`.
```
# Create a learning rate scheduler callback
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch/20)) # traverse a set of learning rate values starting from 1e-4, increasing by 10**(epoch/20) every epoch
```
This allows us to update the learning rate after each epoch. It is useful because it allows us to plot the loss over all learning rate values

* `TensorBoard` - as written by Sol

```
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tensorflow.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
```
Enables visualisation concurrently with the machine learning workflow with tensorboard. In describing what `histogram_freq` the document can be quoted as "frequency (in epochs) at which to compute activation and weight histograms for the layers of the model. If set to 0, histograms won't be computed. Validation data (or split) must be specified for histogram visualizations."

* https://www.tensorflow.org/tensorboard/get_started
* https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard

**Sol's use of ImageGen and tensorboard**
```
history = model.fit(train_generator, epochs=epochs, validation_data=validation_generator, callbacks=[tensorboard_callback])
```

```
%reload_ext tensorboard
%tensorboard --logdir {logs_base_dir} --host localhost
from tensorboard import notebook
#notebook.list()
```

### Save the Model

In [ ]:
def save_model(model, folder, file_name):
        # serialize model to JSON
        model_json = model.to_json()
        os.system('mkdir -p {}'.format(folder))
        with open(folder+file_name+'.json', "w") as json_file:
            json_file.write(model_json)

        print("Saved model structure to: %s%s.json"%(folder,file_name))
        # serialize weights to HDF5
        model.save_weights(folder+file_name+".h5")
        print("Saved model weights to: %s%s.h5"%(folder, file_name))

import datetime #module to provide the current date and time for use in the file names
currentDateTime = datetime.datetime.now().strftime("%H-%M_%B-%d-%Y")
modelname = 'model_' + currentDateTime + '_' + optimizer_choice_string + '_' + organ

save_model(model, output_folder, modelname)

# #save model to an eps file
# from keras.utils import plot_model
# eps_file = output_folder + modelname + '.eps'
# plot_model(model,to_file=eps_file,show_shapes=True, show_layer_names=False)
# print("Model saved to eps: " + eps_file)

# #display model
# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot
# SVG(model_to_dot(model, show_shapes=True, show_layer_names=False).create(prog='dot', format='svg'))

### Plot the loss curves

In [ ]:
def plot_loss(history,output_folder):
    fig, ax1 = plt.subplots()
    fig.patch.set_facecolor('white')
    # summarize history for loss
    loss_values = history.history['loss']
    val_loss_values = history.history['val_loss']
    #plt.xlim(left=10)
    loss_values_cropped = loss_values#[10:]
    val_loss_values_cropped = val_loss_values#[10:]
    loss_dif = np.abs(loss_values_cropped - val_loss_values_cropped)
    l1 = ax1.plot(loss_values_cropped,color='cornflowerblue', label='Training loss')
    l2 = ax1.plot(val_loss_values_cropped,color='green', label='Validation loss')
    l_diff = ax1.plot(loss_dif, color='red', label='Absolute loss difference')
    ax1.set_ylabel('Loss', fontsize=18)
    ax1.set_xlabel('Epoch', fontsize=18)
    ax1.legend(loc='best', fontsize=14)
    plt.tick_params(axis='both', which='major', labelsize=14)
    plt.tick_params(axis='both', which='minor', labelsize=8)
    #plt.ylim(0,0.04)
    #plt.xlim(0,410)
    plt.tight_layout()
    
#     currentDateTime = datetime.datetime.now().strftime("%H-%M_%B-%d-%Y")
#     plt.savefig(output_folder + 'acc_loss' + currentDateTime + '_' + optimizer_choice_string + '_' + organ + '.png')
#     np.save(output_folder+'validation_loss_history.npy', np.array(val_loss_values))
#     np.save(output_folder+'training_loss_history.npy', np.array(loss_values))
    plt.show()
    
plot_loss(history, output_folder)

In [ ]:
def plot_loss(history,output_folder):
    fig, ax1 = plt.subplots()
    fig.patch.set_facecolor('white')
    # summarize history for loss
    loss_values = history.history['loss']
    val_loss_values = history.history['val_loss']
    #plt.xlim(left=10)
    loss_values_cropped = loss_values#[10:]
    val_loss_values_cropped = val_loss_values#[10:]
    l1 = ax1.plot(loss_values_cropped,color='cornflowerblue', label='Training loss')
    l2 = ax1.plot(val_loss_values_cropped,color='green', label='Validation loss')
    ax1.set_ylabel('Loss', fontsize=18)
    ax1.set_xlabel('Epoch', fontsize=18)
    ax1.legend(loc='best', fontsize=14)
    plt.tick_params(axis='both', which='major', labelsize=14)
    plt.tick_params(axis='both', which='minor', labelsize=8)
    plt.ylim(0,0.08)
    #plt.xlim(0,410)
    plt.tight_layout()
    
#     currentDateTime = datetime.datetime.now().strftime("%H-%M_%B-%d-%Y")
#     plt.savefig(output_folder + 'acc_loss' + currentDateTime + '_' + optimizer_choice_string + '_' + organ + '_shorter_range_1.png')
#     np.save(output_folder+'validation_loss_history.npy', np.array(val_loss_values))
#     np.save(output_folder+'training_loss_history.npy', np.array(loss_values))
    plt.show()
    
plot_loss(history, output_folder)
    

In [ ]:
def plot_loss(history,output_folder):
    fig, ax1 = plt.subplots()
    fig.patch.set_facecolor('white')
    # summarize history for loss
    loss_values = history.history['loss']
    val_loss_values = history.history['val_loss']
    #plt.xlim(left=10)
    loss_values_cropped = loss_values#[10:]
    val_loss_values_cropped = val_loss_values#[10:]
    l1 = ax1.plot(loss_values_cropped,color='cornflowerblue', label='Training loss')
    l2 = ax1.plot(val_loss_values_cropped,color='green', label='Validation loss')
    ax1.set_ylabel('Loss', fontsize=18)
    ax1.set_xlabel('Epoch', fontsize=18)
    ax1.legend(loc='best', fontsize=14)
    plt.tick_params(axis='both', which='major', labelsize=14)
    plt.tick_params(axis='both', which='minor', labelsize=8)
    plt.ylim(0,0.04)
    #plt.xlim(0,410)
    plt.tight_layout()
    
#     currentDateTime = datetime.datetime.now().strftime("%H-%M_%B-%d-%Y")
#     plt.savefig(output_folder + 'acc_loss' + currentDateTime + '_' + optimizer_choice_string + '_' + organ + '_shorter_range_2.png')
#     np.save(output_folder+'validation_loss_history.npy', np.array(val_loss_values))
#     np.save(output_folder+'training_loss_history.npy', np.array(loss_values))
    plt.show()
    
plot_loss(history, output_folder)

#### Load a Previous Model

In [ ]:
output_folder = 
os.system('mkdir -p {}'.format(output_folder))

In [ ]:
#Specify the json model file name
#json_filename = "/hepgpu9-data1/sbridger/mphys-2d-network-2019-20/plots/BRAIN_STEM/with_zero_dice_scores/MSD_3D/fourth_root/model_20-27_August-16-2021_RMSprop_BRAIN_STEM.json"
json_filename = 

#Specify the weight h5 filename
weights_filename = 

In [ ]:
#open the json file containing the model
json_file = open(json_filename, 'r')
json_string = json_file.read() #read the json file to a string
json_file.close()
print('json file read and closed')


#load the model from the json string (doesn't include the weights)
loaded_model = tensorflow.keras.models.model_from_json(json_string)
print('model loaded')
#initialise the saved weights
loaded_model.load_weights(weights_filename, by_name=False)
print('weights loaded')

In [ ]:
#### If using a model with custom activation (such as absolute) loading the .json can give problems. Simply define the model architecture and then load the weights (.h5 file) onto it.
model.load_weights(weights_filename, by_name=False)
print('weights loaded')

After running the cells above, the loaded model is stored in the variable "loaded_model". This is done as a way of preventing accidental load of a model as "model", which would overwrite the model one might have trained. If you want to work with a loaded model, please uncomment and run the following cell (I advice you comment it back afterwards):

In [ ]:
model = loaded_model

In [ ]:
output_folder = "/hepgpu9-data1/plots/"+organ+"/after_resizing/single_slices/cartesian/ReLU/"+metric+"/fourth_root/batch_" + str(batch_size)
#Load predictions
os.system('mkdir -p {}'.format(output_folder))
predicted_metric_training = np.load(output_folder + "/predicted_metric_training_"+metric+""+organ+""+str(percentage_of_split)+".npy")
predicted_metric_validation = np.load(output_folder + "/predicted_metric_validation_"+metric+""+organ+""+str(percentage_of_split)+".npy")

### Predict with the Model

In [ ]:
output_folder = 

In [ ]:
predicted_metric_training_name = output_folder + '/predicted_metric_training_' + metric+'_'+organ+'_'+\
                                str(percentage_of_split)
np.save(predicted_metric_training_name, predicted_metric_training)
print("Training predictions array saved")
# Must have 3D_MSD as previously metric2=3D_MSD, now changed to MSD_3D
predicted_metric_validation_name = output_folder + '/predicted_metric_validation_' + metric+'_'+organ+'_'+\
                                str(percentage_of_split)
np.save(predicted_metric_validation_name, predicted_metric_validation)
print("Validation predictions array saved")

In [ ]:
X_train_load = np.load('PAROTID_LT_multiple_train_ref_image_reg_contour_{}_array.npy'.format(x_im), mmap_mode = 'r')[:n_train]
X_validate_load = np.load('PAROTID_LT_multiple_validate_ref_image_reg_contour_{}_array.npy'.format(x_im), mmap_mode = 'r')[:n_validate]
bright=[]
arr_choice = X_train_load
for i in range(len(arr_choice)):
    if np.max(arr_choice[i,:,:,0])>3500:
        bright.append(i)
print(len(bright))
X_bright_val = x_validate[bright]
predicted_metric_validation_bright = model.predict(X_bright_val, batch_size = batch_size, verbose=0)
predicted_validation_bright = np.reshape(predicted_metric_validation_bright, len(X_bright_val))

Y_bright_val = y_validate[bright]
pearson_validation_bright, p_validation_bright = stats.pearsonr(Y_bright_val, predicted_validation_bright)

print(pearson_validation_bright)


In [ ]:
X_train = x_train
X_validate = x_validate
Y_train = y_train
Y_validate = y_validate

predicted_metric_training = model.predict(X_train, batch_size=batch_size, verbose=0)
predicted_metric_validation = model.predict(X_validate, batch_size = batch_size, verbose=0)

predicted_training = np.reshape(predicted_metric_training, len(Y_train))
predicted_validation = np.reshape(predicted_metric_validation, len(Y_validate))

from scipy import stats

pearson_training, p_training = stats.pearsonr(Y_train, predicted_training)
pearson_validation, p_validation = stats.pearsonr(Y_validate, predicted_validation)

print("Pearson training "+metric_choice+": "+str(pearson_training)+"\t p_training "+metric_choice+": "+str(p_training))
print("Pearson validation "+metric_choice+": "+str(pearson_validation)+"\t p_validation "+metric_choice+": "+str(p_validation))

## Results

### Plot Functions

In [ ]:
def plot_Metric(actual_metric_training, actual_metric_validation, predicted_metric_training, predicted_metric_validation, \
              metric_name_choice, output_folder):
    fig, ax1 = plt.subplots()
    fig.patch.set_facecolor('white')
    # summarize history for loss
    plt.scatter(actual_metric_training, predicted_metric_training, marker = 'x', color = 'black', label='Training data')
    plt.scatter(actual_metric_validation, predicted_metric_validation, marker = 'o', color = 'red', label='Validation data')
    ax1.set_ylabel('Predicted ' + metric_choice, fontsize=18)
    ax1.set_xlabel('Actual ' + metric_choice, fontsize=18)
    
    
    bottom, top = plt.ylim()
    left, right = plt.xlim()
    lower = min(bottom, left)
    higher = max(top, right)
    plt.plot([lower, higher], [lower, higher], label='Ideal behaviour')
    
    #ax1.legend(loc='best')
    
    plt.tick_params(axis='both', which='major', labelsize=14)
    plt.tick_params(axis='both', which='minor', labelsize=8)
    plt.tight_layout()

#     currentDateTime = datetime.datetime.now().strftime("%H-%M_%B-%d-%Y")
#     plt.savefig(output_folder + metric + '_graph' + currentDateTime + '_' + optimizer_choice_string + '_' + organ + '.png')
    plt.show()
    
plot_Metric(Y_train, Y_validate, predicted_metric_training, predicted_metric_validation, metric_choice, output_folder)

In [ ]:
def plot_Metric_heatmap(actual_metric, predicted_metric, trainingOrValidation, metric_name_choice, output_folder):
    fig, ax1 = plt.subplots()
    fig.patch.set_facecolor('white')

    # summarize history for loss
#     print(predicted_metric[0].shape)
    predicted_metric_reshaped = np.reshape(predicted_metric, actual_metric.shape)
    h = plt.hist2d(actual_metric, predicted_metric_reshaped, 50, range = [[0,1], [0,1]], density = True, cmap='viridis')
    plt.colorbar(h[3], ax=ax1)
    y_axis_label = 'Predicted ' + trainingOrValidation + ' ' + metric_name_choice
    x_axis_label = 'Actual ' + trainingOrValidation + ' ' + metric_name_choice
    ax1.set_ylabel(y_axis_label, fontsize=18)
    ax1.set_xlabel(x_axis_label, fontsize=18)

    bottom, top = plt.ylim()
    left, right = plt.xlim()
    lower = min(bottom, left)
    higher = max(top, right)
    plt.plot([lower, higher], [lower, higher], color = 'white', label = 'Ideal behaviour')
    leg = ax1.legend(loc='best')
    leg.get_frame().set_alpha(0)
    plt.setp(leg.get_texts(), color='w')

    plt.tick_params(axis='both', which='major', labelsize=14)
    plt.tick_params(axis='both', which='minor', labelsize=8)
    plt.tight_layout()

#     currentDateTime = datetime.datetime.now().strftime("%H-%M_%B-%d-%Y")
#     plt.savefig(output_folder + metric_name_choice + '_heatmap_' + trainingOrValidation + '_' + currentDateTime + '_' \
#                 + optimizer_choice_string + '_' + organ + '.png')
    plt.show()

plot_Metric_heatmap(Y_train, predicted_metric_training, "Training", metric_choice, output_folder)
plot_Metric_heatmap(Y_validate, predicted_metric_validation, "Validation", metric_choice, output_folder)

In [ ]:
import matplotlib.ticker as tik
n_bins0 = 25
def plot_Metric_histogram(actual_metric, predicted_metric, trainingOrValidation, metric_name_choice, output_folder):
    fig, ax1 = plt.subplots()
    fig.patch.set_facecolor('white')
    plt.hist(actual_metric, bins = n_bins, range=(0,1), density = True, alpha=0.6, label='Actual ' + metric_name_choice)
    #density = True makes the first element of the return tuple will be the counts normalized to form a probability density
    #i.e. the area (or integral) under the histogram will sum to 1
    plt.hist(predicted_metric, bins = n_bins, range=(0,1), density = True, alpha=0.6, label='Predicted ' + metric_name_choice)
    plt.yscale('log')
    plt.legend(loc='best')
    ax1.set_ylabel('Frequency', fontsize=18)
    x_axis_label = trainingOrValidation + ' ' + metric_name_choice
    ax1.set_xlabel(x_axis_label, fontsize=18)
    #set significant figures on x axis
    #ax1.xaxis.set_major_formatter(tik.FormatStrFormatter('%0.2f'))
    #locx = tik.MultipleLocator(base=0.1) # this locator puts ticks at regular intervals
    #locy = tik.MultipleLocator(base=1)
    #ax1.xaxis.set_major_locator(locx)
    #ax1.yaxis.set_major_locator(locy)
#     currentDateTime = datetime.datetime.now().strftime("%H-%M_%B-%d-%Y")
    #plt.savefig(output_folder + metric + '_histogram_' + trainingOrValidation + '_' + currentDateTime + '_' +\
    #            optimizer_choice_string + '_' + organ + '.png')
    plt.show()
    
plot_Metric_histogram(Y_train, predicted_metric_training, "Training", metric_choice, output_folder)
plot_Metric_histogram(Y_validate, predicted_metric_validation, "Validation", metric_choice, output_folder) 

In [ ]:
print(Y_train.shape)
print(predicted_metric_training.shape)
Y_train_reshaped = np.reshape(Y_train, predicted_metric_training.shape)
print(Y_train_reshaped.shape)
joint_train = np.hstack((Y_train_reshaped, predicted_metric_training))
joint_dataFrame = pd.DataFrame(joint_train, columns = ["Actual Training " + metric, "Predicted Training " + metric])
sb.set_style("white")
sb.set_style({'xtick.bottom': True,
 'xtick.color': '.15',
 'xtick.direction': 'out',
 'xtick.top': False,
 'ytick.color': '.15',
 'ytick.direction': 'out',
 'ytick.left': True,
 'ytick.right': True,
 'ytick.major.size': 05.0, 
 'xtick.major.size': 5.0})
sb.set_context("notebook", font_scale=1.5)
sb.jointplot(x = "Actual Training " + metric, y = "Predicted Training " + metric, data=joint_dataFrame, \
             stat_func = None)

In [ ]:
print(Y_validate.shape)
print(predicted_metric_validation.shape)
Y_validate_reshaped = np.reshape(Y_validate, predicted_metric_validation.shape)
print(Y_validate_reshaped.shape)
joint_validate = np.hstack((Y_validate_reshaped, predicted_metric_validation))
joint_dataFrame_validate = pd.DataFrame(joint_validate, columns = ["Actual Validation " + metric, \
                                                                   "Predicted Validation " + metric])
sb.set_style("white")
sb.set_style({'xtick.bottom': True,
 'xtick.color': '.15',
 'xtick.direction': 'out',
 'xtick.top': False,
 'ytick.color': '.15',
 'ytick.direction': 'out',
 'ytick.left': True,
 'ytick.right': True,
 'ytick.major.size': 05.0, 
 'xtick.major.size': 5.0})
print(sb.axes_style())
sb.set_context("notebook", font_scale=1.5)
sb.jointplot(x = "Actual Validation " + metric, y = "Predicted Validation " + metric, data=joint_dataFrame_validate, \
             stat_func = None)

### Evaluating Predictions

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error, \
mean_absolute_error, r2_score, max_error, explained_variance_score

R^2 Score

In [ ]:
train_r2_score = r2_score(Y_train, predicted_metric_training)#, multioutput='variance_weighted') 
val_r2_score = r2_score(Y_validate, predicted_metric_validation)#, multioutput='variance_weighted')

print("train_r2_score: ", train_r2_score)
print("val_r2_score: ", val_r2_score)

Mean Absolute Error (MAE)

In [ ]:
train_mae = mean_absolute_error(Y_train, predicted_metric_training)#, multioutput='variance_weighted') 
val_mae = mean_absolute_error(Y_validate, predicted_metric_validation)#, multioutput='variance_weighted')
std_train_mae = np.std(np.abs(Y_train-predicted_metric_training))
std_val_mae = np.std(np.abs(Y_validate-predicted_metric_validation))
print("train_mae ± std: {} ± {}".format(train_mae, std_train_mae))
print("val_mae ± std: {} ± {}".format(val_mae, std_val_mae))

Mean Absolute Percentage Error (MAPE)

In [ ]:
train_mape = mean_absolute_percentage_error(Y_train, predicted_metric_training)#, multioutput='variance_weighted') 
val_mape = mean_absolute_percentage_error(Y_validate, predicted_metric_validation)#, multioutput='variance_weighted')

print("train_mape: ", train_mape)
print("val_mape: ", val_mape)

In [ ]:
metric_scores_df = pd.DataFrame([(pearson_training), (pearson_validation),
                                 (train_r2_score), (val_r2_score),
                                 (train_mae), (val_mae),
                                (std_train_mae), (std_val_mae)],
                               columns=[metric_choice],
                               index=["Pearson training", "Pearson validation",
                                        "r^2 score training", "r^2 score validation",
                                        "MAE training", "MAE validation",
                                     "MAE training std", "MAE validation std"])

# print(metric_scores_df)

fig = plt.figure(figsize = (8, 2))
fig.patch.set_facecolor('white')
ax = fig.add_subplot(111)

ax.table(cellText = metric_scores_df.values.round(decimals=4),
         rowLabels = metric_scores_df.index,
         colLabels = metric_scores_df.columns,
         cellLoc="center", loc = "top",
         bbox=[0, -0.55, 0.4, 1.5] 
         )

ax.set_title(metric_choice+" network", loc="left")
ax.axis("off")
plt.show()

Group the metric arrays by intervals of width 0.1 for plotting

In [ ]:
def sort_by_interval(dataframe):
    lb, ub, metric_idx, sort_metric_idx, sort_idx=[],[],[],[],[]
    for i in range(10):
        lb.append(str(i/10)); ub.append(str(0.1+i/10))
        metric_idx.append(lb[i]+' < '+metric+' <= '+ub[i])
        sort_idx.append(dataframe.query(metric_idx[i]).index.to_numpy(dtype=int))
    sort_metric_idx = np.array(sort_idx, dtype=object)
#     print(sort_metric_idx)
    return sort_metric_idx

# sort_by_interval(pred_val_df)

In [ ]:
def boxplot(metric_array, predicted_metric, dataframe, metric_name_choice, TorV, absolute=None):
    data=[]
    interval_number=10; n=100
    null_intervals=False
    for i in range(interval_number):
        error_array=error(metric_array, predicted_metric, absolute=absolute)
        interval_data=error_array[sort_by_interval(dataframe)[i]]
#         interval_data=interval_data/len(interval_data)
        if 0<len(interval_data)<n :
            print('Interval '+str(i+1)+' contains '+str(len(interval_data))+' data points. Change the interval widths'\
                  ' to not have small datasets.')
        if len(interval_data)<=1:
            null_intervals=True; null_intervals_idx=[]
            null_intervals_idx.append(i)
            del interval_data
        else:
            data.append(interval_data)
    fig, ax0 = plt.subplots()
    fig.patch.set_facecolor('white')
    ax0.boxplot(data)
    ax0.set_title(metric_name_choice+' '+TorV)
    ax0.set_ylabel('Absolute error density')
    ax0.set_xlabel(metric_name_choice+' intervals')
    if null_intervals==False:
        plt.xticks([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], ['(0,0.1]','(0.1,0.2]','(0.2,0.3]','(0.3,0.4]','(0.4,0.5]',
                    '(0.5,0.6]','(0.6,0.7]','(0.7,0.8]','(0.8,0.9]','(0.9,1]'], fontsize=9.5)
    if null_intervals==True:
        plt.xticks([1, 2, 3, 4, 5, 6, 7, 8, 9], ['(0,0.2]','(0.2,0.3]','(0.3,0.4]','(0.4,0.5]',
                    '(0.5,0.6]','(0.6,0.7]','(0.7,0.8]','(0.8,0.9]','(0.9,1]'], fontsize=9.5)
    ax0.set_ylim(0,0.16)

    plt.show()

boxplot(Y_train, predicted_metric_training, pred_train_df, metric_name, 'training', absolute=True)
boxplot(Y_validate, predicted_metric_validation, pred_val_df, metric_name, 'validation', absolute=True)

### Error Plot Functions

In [ ]:
def error(metric_array, metric_prediction, absolute=None):   
    temp=[]
    sq_metric_prediction = np.squeeze(metric_prediction)
    for i in range(len(metric_array)):
        if absolute==True:
            error = np.abs(metric_array[i] - sq_metric_prediction[i])
        if absolute==False:
            error = metric_array[i] - sq_metric_prediction[i]
        temp.append(error)
        error_array = np.array(temp)
    return error_array

In [ ]:
pred_val_df = pd.DataFrame({metric+' predicted validation': np.squeeze(predicted_metric_validation),
                            metric: Y_validate,
                            metric+' absolute error': error(Y_validate, predicted_metric_validation, absolute=True)},
                           index=np.arange(len(predicted_metric_validation)),
                           columns=[metric+' predicted validation', metric,
                                    metric+' absolute error'])

pred_train_df = pd.DataFrame({metric+' predicted training': np.squeeze(predicted_metric_training),
                              metric: Y_train,
                              metric+' absolute error': error(Y_train, predicted_metric_training, absolute=True)},
                             index=np.arange(len(predicted_metric_training)),
                             columns=[metric+' predicted training', metric,
                                      metric+' absolute error'])

print(pred_val_df)
print(pred_train_df)

In [ ]:
def AE_metric_plot(metric_array, metric_prediction, metric_name_choice, TorV):
    fig, ax0 = plt.subplots()
    error_array = error(metric_array, metric_prediction, absolute=True)
    plt.scatter(metric_array, error_array)
    fig.patch.set_facecolor('white')
    ax0.set_xlabel(metric_name_choice+" "+TorV+' score', fontsize=18)
    ax0.set_ylabel('Absolute error', fontsize=18)
    plt.tight_layout()
#     ax0.set_xlim(-0.01, 1.01)
#     ax0.set_ylim(-0.01)
    plt.show()

In [ ]:
def error_hist(metric_array, metric_prediction, metric_name_choice, TorV):
    fig, ax1 = plt.subplots(figsize=(5, 3.5))
#     absolute lets you choose if you want the error or the absolute error 
    error_array = error(metric_array, metric_prediction, absolute=True) 
    print('mean ± std MAE: {} ± {}'.format(np.mean(error_array), np.std(error_array)))
    fig.patch.set_facecolor('white')
    metric_freq_histog, bin_edges = np.histogram(metric_array, bins=40)
    print(metric_freq_histog)
    plt.hist()
    plt.hist(error_array, density = False, bins = 40)#200, range = (-1,1))
    ax1.set_ylabel('Frequency', fontsize=18)
    ax1.set_xlabel(metric_choice + " " + TorV + ' absolute error', fontsize=18)
    plt.tick_params(axis='both', which='major', labelsize=14)
#         totalNumberSlices = str(metric_array[i].shape[0])
    plt.tight_layout()
    plt.show()

# error_hist(Y_train, predicted_metric_training, metric_name, 'training')
# error_hist(Y_validate, predicted_metric_validation, metric_name, 'validation')

def error_freq_hist(metric_array, metric_prediction, metric_name_choice, TorV, Nbins):
    fig, ax1 = plt.subplots(figsize=(5, 3.5))
#     absolute lets you choose if you want the error or the absolute error 
    error_array = error(metric_array, metric_prediction, absolute=True) 
    print('mean ± std MAE: {} ± {}'.format(np.mean(error_array), np.std(error_array)))
    fig.patch.set_facecolor('white')
    metric_freq_histog, bin_edges = np.histogram(metric_array, bins=Nbins)
    print(len(metric_freq_histog))
    print(len(bin_edges))
    MAEs=np.empty((len(metric_freq_histog)))
    
    cumu_sum=metric_freq_histog[0]
    for i in range(len(MAEs)):
        bin_i = metric_freq_histog[i]
        bin_before_i = metric_freq_histog[i-1]
        if i==0:
            MAEs[i] = np.max(error_array[:cumu_sum])
        else:
            MAEs[i] = a
            cumu_sum+=bin_i
    ax1.scatter(bin_edges[:-1], MAEs)
    plt.xlim(0,1)
    ax1.set_ylabel('Dice validation MAE', fontsize=18)
    ax1.set_xlabel(metric_choice + ' score', fontsize=18)
    plt.tick_params(axis='both', which='major', labelsize=14)
    plt.tight_layout()
    plt.show()

error_freq_hist(Y_validate[:9326], predicted_metric_validation[:9326], metric_name, 'validation', 2000)

In [ ]:
def error_metric_hist2d(metric_array, metric_prediction, metric_name_choice, TorV, bin_x, bin_y):
    fig, ax1 = plt.subplots()
    n_binsx = bin_x; n_binsy=bin_y
#     absolute lets you choose if you want the error or the absolute error 
    error_array = error(metric_array, metric_prediction, absolute=True) 
    fig.patch.set_facecolor('white')
    plt.hist2d(metric_array, error_array, bins=(n_binsx, n_binsy), cmap=plt.cm.jet)
    ax1.set_xlabel(metric_name_choice+' score', fontsize=18)
    ax1.set_ylabel(metric_name_choice + " " + TorV + ' absolute error', fontsize=18)
    plt.tick_params(axis='both', which='major', labelsize=14)
    plt.colorbar(label='Frequency')
    plt.tight_layout()
    plt.show()

error_metric_hist2d(Y_validate[:9326], predicted_metric_validation[:9326], 'Dice', 'validation', 30, 30)
# error_metric_hist2d(Y_train1, predicted_metric_training[0], metric_name[0], 'training')
# error_metric_hist2d(Y_validate1, predicted_metric_validation[0], metric_name[0], 'validation')
# error_metric_hist2d(Y_train2, predicted_metric_training[1], metric_name[1], 'training')
# error_metric_hist2d(Y_validate2, predicted_metric_validation[1], metric_name[1], 'validation')

### More Prediction Evaluation

In [ ]:
%%capture cap --no-stderr
#Calculate mean and largest deviations of prediction from actual value

#The prediction arrays have shape (length, 1). We want to reshape them to just (length,)
predicted_metric_training_newshape = np.reshape(predicted_metric_training, len(Y_train))
predicted_metric_validation_newshape = np.reshape(predicted_metric_validation, len(Y_validate))
mean_difference_training = np.sum(abs(predicted_metric_training_newshape - Y_train))/len(Y_train)
largest_difference_training = abs(predicted_metric_training_newshape - Y_train).max()
mean_difference_validation = np.sum(abs(predicted_metric_validation_newshape - Y_validate))/len(Y_validate)
largest_difference_validation = abs(predicted_metric_validation_newshape - Y_validate).max()
print('Average absolute difference in '+ metric+', training data : ')
print(mean_difference_training)
print('Average absolute difference in '+ metric+', validation data : ')
print(mean_difference_validation)
print('Largest absolute difference in '+ metric+', training data : ')
print(largest_difference_training)
print('Largest absolute difference in '+ metric+', validation data : ')
print(largest_difference_validation)

#Calculate percentiles
q = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 96, 97, 98, 99, 100]
training_percentiles = np.percentile(abs(predicted_metric_training_newshape - Y_train), q)
print('The percentiles for the training data are:')
for i in range(0, len(q)):
    print(str(q[i]) + '%: ' + str(training_percentiles[i]))

validation_percentiles = np.percentile(abs(predicted_metric_validation_newshape - Y_validate), q)
print('\nThe percentiles for the validation data are:')
for i in range(0, len(q)):
    print(str(q[i]) + '%: ' + str(validation_percentiles[i]))

In [ ]:
with open(output_folder+'deviations_from_actual_values.txt', 'w') as f:
    f.write(cap.stdout)

### Prediction - Actual DICE score

#### Training Data

In [ ]:
training_metric_difference = predicted_metric_training_newshape - Y_train
fig, ax1 = plt.subplots()
#removes the grey background to the plot
fig.patch.set_facecolor('white')
plt.hist(training_metric_difference, density = False, bins = 200, range = (-1,1))
ax1.set_ylabel('Frequency', fontsize=18)
ax1.set_xlabel('Predicted Training '+metric+' - Actual Training '+metric, fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=14)
currentDateTime = datetime.datetime.now().strftime("%H-%M_%B-%d-%Y")
plt.savefig(output_folder + 'training_'+metric+'_difference_histogram_' + organ + '_' + currentDateTime + '.png')
plt.show()

#### Validation Data

In [ ]:
#Plot histogram of predicted - actual Dice score for validation data
validation_metric_difference = predicted_metric_validation_newshape - Y_validate
fig, ax1 = plt.subplots()
#removes the grey background to the plot
fig.patch.set_facecolor('white')
plt.hist(validation_metric_difference, density = False, bins = 200, range = (-1,1))
ax1.set_ylabel('Frequency', fontsize=18)
ax1.set_xlabel('Predicted Validation '+metric+' - Actual Validation '+metric, fontsize=18)
plt.tick_params(axis='both', which='major', labelsize=14)
currentDateTime = datetime.datetime.now().strftime("%H-%M_%B-%d-%Y")
#plt.savefig(output_folder + 'validation_'+metric+'_difference_histogram_' + organ + '_' + currentDateTime + '.png')
plt.show()

## Discussion

### Introduction

Below will follow:

* Dice to 3D MSD correspondence
* Deviations against actual values of metric
* False positives and negatives
These are useful to analyse which is the best threshold value of the surface distance metric. Any registrations with predicted value above the threshold would be flagged for manual review in a clinical setting.

### Calculate Dice score to 3D MSD correspondence

#### With a Single 3D MSD Array 

In [ ]:
```
from scipy import stats import decimal

x_axis = [] corresponding_mean_dice = [] corresponding_mean_error = [] number_of_datapoints = []

dice_array = multiple_dice_array[np.where(multiple_dice_array != 0)] msd_array = metric_array

if len(dice_array) != len(msd_array): raise ValueError("The dice and msd arrays did not have the same length.")

actual_value = decimal.Decimal('0.0') while (actual_value < decimal.Decimal('1.0')):
```
----------------
```
indices = np.where((actual_value <= msd_array) & (msd_array < actual_value + decimal.Decimal('0.1')))

print(str(actual_value) + " to " + str(actual_value + decimal.Decimal('0.1')) + ": " + str(len(indices[0])))
number_of_datapoints.append(len(indices[0]))

corresponding_mean_dice.append(np.mean(dice_array[indices]))
sd = np.std(dice_array[indices])
corresponding_mean_error.append(sd)#/(len(dice_array[indices])-1)**0.5) #stats.sem(dice_array[indices]))

x_axis.append(actual_value+decimal.Decimal('0.05'))

actual_value += decimal.Decimal('0.1')
```

    
    

#### With Two 3D MSD Arrays at the Same Time

In [ ]:
```
import decimal

x_axis = [] corresponding_mean_dice = [] corresponding_mean_error = [] number_values = [] corresponding_mean_dice_transformed = [] corresponding_mean_error_transformed = [] number_values_transformed = []

dice_array = multiple_dice_array[np.where(multiple_dice_array != 0)] msd_array_transformed = metric_array msd_array = msd_no_zero_dice_normalised

if (len(dice_array) != len(msd_array) or len(dice_array) != len(msd_array_transformed)): raise ValueError("The dice and msd arrays did not have the same length.")

actual_value = decimal.Decimal('0.0') while (actual_value < decimal.Decimal('1.0')):
```
--------------------
```
indices_transformed = np.where((actual_value <= msd_array_transformed) & (msd_array_transformed < actual_value + decimal.Decimal('0.1')))
indices = np.where((actual_value <= msd_array) & (msd_array < actual_value + decimal.Decimal('0.1')))

print(str(actual_value) + " to " + str(actual_value + decimal.Decimal('0.1')) + ": " + str(len(indices[0])) + " " + str(len(indices_transformed[0])))
number_values.append(len(indices[0]))
number_values_transformed.append(len(indices_transformed[0]))

corresponding_mean_dice_transformed.append(np.mean(dice_array[indices_transformed]))
corresponding_mean_error_transformed.append(np.std(dice_array[indices_transformed]))

corresponding_mean_dice.append(np.mean(dice_array[indices]))
corresponding_mean_error.append(np.std(dice_array[indices]))

x_axis.append(actual_value+decimal.Decimal('0.05'))

actual_value += decimal.Decimal('0.1')
```

Plotting Functions for these

In [ ]:
```
fig, ax1 = plt.subplots() fig.patch.set_facecolor('white')

h = plt.hist2d(metric_array, dice_array, 50, range = [[0,1], [0,1]], density = True, cmap='viridis') plt.colorbar(h[3], ax=ax1) y_axis_label = 'Dice score' x_axis_label = '3D MSD fourth-root' ax1.set_ylabel(y_axis_label, fontsize=18) ax1.set_xlabel(x_axis_label, fontsize=18)

plt.tick_params(axis='both', which='major', labelsize=14) plt.tick_params(axis='both', which='minor', labelsize=8) plt.tight_layout()
```

In [ ]:
```
fig, ax1 = plt.subplots() fig.patch.set_facecolor('white') l2 = ax1.errorbar(x_axis, corresponding_mean_dice, corresponding_mean_error, color='b', label="Normal") l1 = ax1.errorbar(x_axis, corresponding_mean_dice_transformed, corresponding_mean_error_transformed, color='g', label="Fourth-root")

l3 = ax1.scatter(x_axis, corresponding_mean_dice, s = 0.1np.array(number_values), color='r') l4 = ax1.scatter(x_axis, corresponding_mean_dice_transformed, s = 0.1np.array(number_values_transformed), color='r')

ax1.set_ylabel('Mean Dice score', fontsize=18) ax1.set_xlabel('3D MSD', fontsize=18) plt.tick_params(axis='both', which='major', labelsize=14) plt.tick_params(axis='both', which='minor', labelsize=8)

#Colour legend markers = [l2, l1] #labels = ["Normal", "Fourth-root"] fig.legend(handles=markers, loc=[0.1111,0.13], numpoints=1)

#Size legend handles, labels = l3.legend_elements(prop="sizes", num = [100,300,500,700], alpha=0.5, color='red') ax1.legend(handles, labels, loc="best", title="#Datapoints", numpoints = 1, labelspacing=1) ax1.xaxis.set_ticks(np.arange(0, 1.1, 0.1)) ax1.yaxis.set_ticks(np.arange(0, 1.1, 0.1)) plt.ylim(0,1)#np.array([all_mean_deviations_training,all_mean_deviations_validation]).max()+0.005) plt.xlim(0,1) plt.tight_layout()
```

In [ ]:
```
ig, ax1 = plt.subplots() fig.patch.set_facecolor('white') l1 = ax1.errorbar(metric_array, dice_array, fmt='.', color='blue') ax1.set_ylabel('Dice score', fontsize=18) ax1.set_xlabel('3D MSD fourth-root', fontsize=18) plt.tick_params(axis='both', which='major', labelsize=14) plt.tick_params(axis='both', which='minor', labelsize=8) start, end = ax1.get_xlim() ax1.xaxis.set_ticks(np.arange(0, 1.1, 0.1)) plt.ylim(-0.01,1)#np.array([all_mean_deviations_training,all_mean_deviations_validation]).max()+0.005) #plt.xlim(-0.05,0.95) plt.tight_layout()
```

### Investigate deviations/error against actual values of the metric

#### Absolute Deviation

`predicted_training = np.reshape(predicted_metric_training, len(Y_train)) predicted_validation = np.reshape(predicted_metric_validation, len(Y_validate))`

In [ ]:
```
import decimal from scipy import stats #To use stats.sem() to calculate the standard error in the mean of the values of an array

x_axis = [] all_mean_deviations_training = [] all_errors_mean_training = []

all_mean_deviations_validation = [] all_errors_mean_validation = []

all_percentile_95_deviations_training = [] all_percentile_95_deviations_validation = []

actual_value = decimal.Decimal('0.1') while (actual_value < decimal.Decimal('1.0')):
    
```
-------------------
```
indices_training = np.where((actual_value <= Y_train) & (Y_train < actual_value + decimal.Decimal('0.05')))
indices_validation = np.where((actual_value <= Y_validate) & (Y_validate < actual_value + decimal.Decimal('0.05')))

print(str(actual_value) + " to " + str(actual_value + decimal.Decimal('0.05')) + ": " + str(len(indices_training[0])) + " " + str(len(indices_validation[0])))

all_mean_deviations_training.append(np.mean(abs(predicted_training[indices_training] - Y_train[indices_training])))
all_errors_mean_training.append(np.std(abs(predicted_training[indices_training] - Y_train[indices_training])))

all_mean_deviations_validation.append(np.mean(abs(predicted_validation[indices_validation] - Y_validate[indices_validation])))
all_errors_mean_validation.append(np.std(abs(predicted_validation[indices_validation] - Y_train[indices_validation])))

all_percentile_95_deviations_training.append(np.percentile(abs(predicted_training[indices_training] - Y_train[indices_training]), 95))
all_percentile_95_deviations_validation.append(np.percentile(abs(predicted_validation[indices_validation] - Y_validate[indices_validation]), 95))

x_axis.append(actual_value + decimal.Decimal('0.025'))

actual_value += decimal.Decimal('0.05')
```

Plotting Code for above

```
fig, ax1 = plt.subplots() fig.patch.set_facecolor('white') #l1 = ax1.errorbar(x_axis, all_mape_training, all_errorbars_mape_training, color='cornflowerblue', label="Training") l2 = ax1.errorbar(x_axis, all_mean_deviations_validation, all_errors_mean_validation, color='g', label="Validation")

#l3 = ax1.scatter(x_axis, all_mape_training, s = 0.1*np.array(datapoints_training), color='r') l4 = ax1.scatter(x_axis, all_mean_deviations_validation, s = np.array(datapoints_validation), color='r')

#Colour legend #fig.legend(handles=[l1,l2], loc=[0.1111,0.13], numpoints=1)

#Size legend handles, labels = l4.legend_elements(prop="sizes", num = [100,250,500,1000], alpha = 0.5, color='red') ax1.legend(handles, labels, loc="best", title="#Datapoints", numpoints = 1, labelspacing=1.1)

ax1.set_ylabel('Mean Absolute Error', fontsize=18) ax1.set_xlabel('3D MSD fourth-root', fontsize=18) plt.tick_params(axis='both', which='major', labelsize=15) plt.tick_params(axis='both', which='minor', labelsize=8) ax1.xaxis.set_ticks(np.arange(0, 1.1, 0.1)) #ax1.yaxis.set_ticks(np.arange(0, 1.1, 0.1)) #plt.grid() plt.ylim(0,0.5)#np.array([all_mean_deviations_training,all_mean_deviations_validation]).max()+0.005) plt.xlim(0.1,1) plt.tight_layout()
```

```
fig, ax1 = plt.subplots() fig.patch.set_facecolor('white') #l1 = ax1.errorbar(x_axis, all_mape_training, all_errorbars_mape_training, color='cornflowerblue', label="Training") #l2 = ax1.errorbar(x_axis, all_mape_validation, all_errorbars_mape_validation, color='g', label="Validation")

#l3 = ax1.scatter(x_axis, all_mape_training, s = 0.1*np.array(datapoints_training), color='r') l4 = ax1.scatter(x_axis, all_mean_deviations_validation, marker='x', s=500, color='r', linewidth=3)

#Colour legend #fig.legend(handles=l2, loc=[0.1111,0.13], numpoints=1)

#ax1.set_ylabel('Mean Absolute Percentage Error', fontsize=18) #ax1.set_xlabel('3D MSD fourth-root', fontsize=18) plt.tick_params(axis='both', which='major', labelsize=25) plt.tick_params(axis='both', which='minor', labelsize=8) ax1.xaxis.set_ticks(np.arange(0, 1.1, 0.1)) ax1.xaxis.tick_top() ax1.yaxis.tick_right() ax1.yaxis.set_ticks(np.arange(0.02, 0.051, 0.005)) plt.grid() plt.ylim(0.02,0.053)#np.array([all_mean_deviations_training,all_mean_deviations_validation]).max()+0.005) plt.xlim(0.2,0.5) plt.tight_layout()
```

```
fig, ax1 = plt.subplots() fig.patch.set_facecolor('white') l1 = ax1.errorbar(x_axis, all_mean_deviations_training, all_errors_mean_training, fmt='.', color='cornflowerblue', label='Training') l2 = ax1.errorbar(x_axis, all_mean_deviations_validation, all_errors_mean_validation, fmt='.', color='green', label='Validation') ax1.set_ylabel('Mean absolute deviation', fontsize=18) ax1.set_xlabel('Actual value', fontsize=18) ax1.legend(loc='best', fontsize=14) plt.tick_params(axis='both', which='major', labelsize=14) plt.tick_params(axis='both', which='minor', labelsize=8) ax1.xaxis.set_ticks(np.arange(0, 1.1, 0.1)) #ax1.yaxis.set_ticks(np.arange(0, 1.1, 0.1)) plt.ylim(0,0.12) #np.array([all_mean_deviations_training,all_mean_deviations_validation]).max()+0.005) plt.xlim(0.1,1) plt.tight_layout()
```


```
fig, ax1 = plt.subplots() fig.patch.set_facecolor('white') l1 = ax1.scatter(x_axis, all_percentile_95_deviations_training,color='cornflowerblue', label='Training deviation') l2 = ax1.scatter(x_axis, all_percentile_95_deviations_validation,color='red', label='Validation deviation') ax1.set_ylabel('95 percentile deviation', fontsize=18) ax1.set_xlabel('Actual value', fontsize=18) ax1.legend(loc='best', fontsize=14) plt.tick_params(axis='both', which='major', labelsize=14) plt.tick_params(axis='both', which='minor', labelsize=8) ax1.xaxis.set_ticks(np.arange(0, 1.1, 0.1)) #plt.ylim(0,np.array([all_percentile_95_deviations_training,all_percentile_95_deviations_validation]).max()+0.005) plt.xlim(0.1,1) plt.tight_layout()
```


#### Mean Absolute Percentage Error (MAPE)

Eliminate all actual values of 0 since would give nan when dividing predicted_training_nozero = predicted_training[np.where(Y_train != 0)] predicted_validation_nozero = predicted_validation[np.where(Y_validate != 0)]

actual_train = Y_train[np.where(Y_train != 0)] actual_validate = Y_validate[np.where(Y_validate != 0)]

```
print("Actual train, actual validate, predictions train, predictions validate") print("Shapes:\n", actual_train.shape, actual_validate.shape, predicted_training_nozero.shape, predicted_validation_nozero.shape) print("Maxima:\n",actual_train.max(), actual_validate.max(), predicted_training_nozero.max(), predicted_validation_nozero.max()) print("Minima:\n",actual_train.min(), actual_validate.min(), predicted_training_nozero.min(), predicted_validation_nozero.min())
```


Work with 3D MSD by transforming back from 3D MSD fourth-root min_value_after_first_root = 0.22832586673831098 max_value_after_first_root = 2.7136906443840783
```
predicted_training_nozero = ((max_value_after_first_root - min_value_after_first_root)predicted_training_nozero*2 + min_value_after_first_root)2 predicted_validation_nozero = ((max_value_after_first_root - min_value_after_first_root)*predicted_validation_nozero2 + min_value_after_first_root)**2

actual_train = ((max_value_after_first_root - min_value_after_first_root)actual_train*2 + min_value_after_first_root)2 actual_validate = ((max_value_after_first_root - min_value_after_first_root)*actual_validate2 + min_value_after_first_root)**2
```

```
import decimal from scipy import stats #To use stats.sem() which provides the standard error in the mean of the values of an array

x_axis = [] all_mape_training = [] all_errorbars_mape_training = [] datapoints_training = [] all_mape_validation = [] all_errorbars_mape_validation = [] datapoints_validation = []

if (len(actual_train) != len(predicted_training_nozero) or len(actual_validate) != len(predicted_validation_nozero)): raise ValueError("The arrays of actual values and predictions did not have the same shape!")

actual_value = decimal.Decimal('0.0') while (actual_value < decimal.Decimal('1.0')):
``` 
-------------------------------------
```
indices_training = np.where((actual_value <= actual_train) & (actual_train < actual_value + decimal.Decimal('0.1')))
    indices_validation = np.where((actual_value <= actual_validate) & (actual_validate < actual_value + decimal.Decimal('0.1')))
    
    print(str(actual_value) + " to " + str(actual_value + decimal.Decimal('0.1')) + ": " + str(len(indices_training[0])) + " " + str(len(indices_validation[0])))
    
    mape_training = np.mean(abs(actual_train[indices_training]-predicted_training_nozero[indices_training])/actual_train[indices_training])
    all_errorbars_mape_training.append(np.std(abs(actual_train[indices_training]-predicted_training_nozero[indices_training])/actual_train[indices_training]))
    all_mape_training.append(mape_training)
    datapoints_training.append(len(indices_training[0]))
    
    mape_validation = np.mean(abs(actual_validate[indices_validation]-predicted_validation_nozero[indices_validation])/actual_validate[indices_validation])
    all_errorbars_mape_validation.append(np.std(abs(actual_validate[indices_validation]-predicted_validation_nozero[indices_validation])/actual_validate[indices_validation]))
    all_mape_validation.append(mape_validation)
    datapoints_validation.append(len(indices_validation[0]))
    
    x_axis.append(actual_value+decimal.Decimal('0.05'))
    
    actual_value += decimal.Decimal('0.1')
```


Plotting the above

```
fig, ax1 = plt.subplots() fig.patch.set_facecolor('white') #l1 = ax1.errorbar(x_axis, all_mape_training, all_errorbars_mape_training, color='cornflowerblue', label="Training") l2 = ax1.errorbar(x_axis, all_mape_validation, all_errorbars_mape_validation, color='g', label="Validation")

#l3 = ax1.scatter(x_axis, all_mape_training, s = 0.1np.array(datapoints_training), color='r') l4 = ax1.scatter(x_axis, all_mape_validation, s = 0.5np.array(datapoints_validation), color='r')

#Colour legend #fig.legend(handles=[l1,l2], loc=[0.1111,0.13], numpoints=1)

#Size legend handles, labels = l4.legend_elements(prop="sizes", num = [50,100,250,500,1000], alpha = 0.5, color='red') ax1.legend(handles, labels, loc="best", title="#Datapoints", numpoints = 1, labelspacing=1)

ax1.set_ylabel('Mean Absolute Percentage Error', fontsize=18) ax1.set_xlabel('3D MSD fourth-root', fontsize=18) plt.tick_params(axis='both', which='major', labelsize=15) plt.tick_params(axis='both', which='minor', labelsize=8) ax1.xaxis.set_ticks(np.arange(0, 1.1, 0.1)) #ax1.yaxis.set_ticks(np.arange(0, 1.1, 0.1)) plt.grid() plt.ylim(0,0.6)#np.array([all_mean_deviations_training,all_mean_deviations_validation]).max()+0.005) plt.xlim(0,1) plt.tight_layout()
```

```
fig, ax1 = plt.subplots() fig.patch.set_facecolor('white') #l1 = ax1.errorbar(x_axis, all_mape_training, all_errorbars_mape_training, color='cornflowerblue', label="Training") #l2 = ax1.errorbar(x_axis, all_mape_validation, all_errorbars_mape_validation, color='g', label="Validation")

#l3 = ax1.scatter(x_axis, all_mape_training, s = 0.1*np.array(datapoints_training), color='r') l4 = ax1.scatter(x_axis, all_mape_validation, marker='x', s=500, color='r', linewidth=3)

#Colour legend #fig.legend(handles=l2, loc=[0.1111,0.13], numpoints=1)

#ax1.set_ylabel('Mean Absolute Percentage Error', fontsize=18) #ax1.set_xlabel('3D MSD fourth-root', fontsize=18) plt.tick_params(axis='both', which='major', labelsize=25) plt.tick_params(axis='both', which='minor', labelsize=8) ax1.xaxis.set_ticks(np.arange(0, 1.1, 0.1)) ax1.xaxis.tick_top() ax1.yaxis.tick_right() #ax1.yaxis.set_ticks(np.arange(0, 1.1, 0.1)) plt.grid() plt.ylim(0.025,0.14)#np.array([all_mean_deviations_training,all_mean_deviations_validation]).max()+0.005) plt.xlim(0.25,0.71) plt.tight_layout()
```

```
fig, ax1 = plt.subplots() fig.patch.set_facecolor('white') l1 = ax1.errorbar(Y_validate, abs(Y_validate-predicted_validation)/Y_validate, fmt='.', color='blue') ax1.set_ylabel('APE', fontsize=18) ax1.set_xlabel('3D MSD fourth-root', fontsize=18) plt.tick_params(axis='both', which='major', labelsize=14) plt.tick_params(axis='both', which='minor', labelsize=8) start, end = ax1.get_xlim() ax1.xaxis.set_ticks(np.arange(0, 1.1, 0.1)) plt.ylim(-0.01,1)#np.array([all_mean_deviations_training,all_mean_deviations_validation]).max()+0.005) #plt.xlim(-0.05,0.95) plt.tight_layout()
```


#### Symmetric Mean Absolute Percentage Error (SMAPE)

Eliminate all entries at which both actual value and prediction are 0 since would give nan when dividing unwanted_training_entries = np.where((Y_train == 0) & (predicted_training == 0)) unwanted_validation_entries = np.where((Y_validate == 0) & (predicted_validation == 0))
```
predicted_training_nozero = np.delete(predicted_training, unwanted_training_entries) predicted_validation_nozero = np.delete(predicted_validation, unwanted_validation_entries)
actual_training = np.delete(Y_train, unwanted_training_entries) actual_validate = np.delete(Y_validate, unwanted_validation_entries)
```

```
x_axis = [] all_smape_training = [] all_errorbars_smape_training = [] all_smape_validation = [] all_errorbars_smape_validation = []

if (len(actual_train) != len(predicted_training_nozero) or len(actual_validate) != len(predicted_validation_nozero)): raise ValueError("The arrays of actual values and predictions did not have the same shape!")

actual_value = decimal.Decimal('0.0') while (actual_value < decimal.Decimal('1.0')):
```
-------------
```
indices_training = np.where((actual_value <= actual_train) & (actual_train < actual_value + decimal.Decimal('0.1')))
indices_validation = np.where((actual_value <= actual_validate) & (actual_validate < actual_value + decimal.Decimal('0.1')))

print(str(actual_value) + " to " + str(actual_value + decimal.Decimal('1')) + ": " + str(len(indices_training[0])) + " " + str(len(indices_validation[0])))

all_smape_training.append(np.mean(abs(actual_train[indices_training]-predicted_training_nozero[indices_training])/(abs(actual_train[indices_training])+abs(predicted_training_nozero[indices_training]))))
all_errorbars_smape_training.append(stats.sem(abs(actual_train[indices_training]-predicted_training_nozero[indices_training])/(abs(actual_train[indices_training])+abs(predicted_training_nozero[indices_training]))))

all_smape_validation.append(np.mean(abs(actual_validate[indices_validation]-predicted_validation_nozero[indices_validation])/(abs(actual_validate[indices_validation])+abs(predicted_validation_nozero[indices_validation]))))
all_errorbars_smape_validation.append(stats.sem(abs(actual_validate[indices_validation]-predicted_validation_nozero[indices_validation])/(abs(actual_validate[indices_validation])+abs(predicted_validation_nozero[indices_validation]))))

x_axis.append(actual_value+decimal.Decimal('0.05'))

actual_value += decimal.Decimal('0.1')
```

```
fig, ax1 = plt.subplots() fig.patch.set_facecolor('white') l1 = ax1.errorbar(x_axis, all_smape_training, all_errorbars_smape_training, fmt='.',color='cornflowerblue', label="Training") l2 = ax1.errorbar(x_axis, all_smape_validation, all_errorbars_smape_validation, fmt='.',color='g', label="Validation") ax1.set_ylabel('Symmetric Mean Absolute Percentage Error', fontsize=18) ax1.set_xlabel('3D MSD fourth-root', fontsize=18) plt.tick_params(axis='both', which='major', labelsize=14) plt.tick_params(axis='both', which='minor', labelsize=8) ax1.legend(loc='best', fontsize=14, numpoints=1) ax1.xaxis.set_ticks(np.arange(0, 1.1, 0.1)) #ax1.yaxis.set_ticks(np.arange(0, 1.1, 0.1)) #plt.ylim(0,0.15)#np.array([all_mean_deviations_training,all_mean_deviations_validation]).max()+0.005) #plt.xlim(0.1,1) plt.tight_layout()
```

In [ ]:
# THE REST OF THE CODE IS STILL WORK IN PROGRESS

#### Mean Arctangent Absolute Percentage Error

```
#Eliminate all entries at which both actual value and prediction are 0 since would give nan when dividing unwanted_training_entries = np.where((Y_train == 0) & (predicted_training == 0)) unwanted_validation_entries = np.where((Y_validate == 0) & (predicted_validation == 0))

predicted_training_nozero = np.delete(predicted_training, unwanted_training_entries) predicted_validation_nozero = np.delete(predicted_validation, unwanted_validation_entries)

actual_training = np.delete(Y_train, unwanted_training_entries) actual_validate = np.delete(Y_validate, unwanted_validation_entries)
```


```
x_axis = [] all_maape_training = [] all_errorbars_maape_training = [] all_maape_validation = [] all_errorbars_maape_validation = []

if (len(actual_train) != len(predicted_training_nozero) or len(actual_validate) != len(predicted_validation_nozero)): raise ValueError("The arrays of actual values and predictions did not have the same shape!")

actual_value = decimal.Decimal('0.0') while (actual_value < decimal.Decimal('1.0')):
```

---------------

```
indices_training = np.where((actual_value <= actual_train) & (actual_train < actual_value + decimal.Decimal('0.1')))
indices_validation = np.where((actual_value <= actual_validate) & (actual_validate < actual_value + decimal.Decimal('0.1')))

print(str(actual_value) + " to " + str(actual_value + decimal.Decimal('1')) + ": " + str(len(indices_training[0])) + " " + str(len(indices_validation[0])))

all_maape_training.append(np.mean(np.arctan(abs(actual_train[indices_training]-predicted_training_nozero[indices_training])/actual_train[indices_training])))
all_errorbars_maape_training.append(stats.sem(np.arctan(abs(actual_train[indices_training]-predicted_training_nozero[indices_training])/actual_train[indices_training])))

all_maape_validation.append(np.mean(abs(np.arctan(actual_validate[indices_validation]-predicted_validation_nozero[indices_validation])/actual_validate[indices_validation])))
all_errorbars_maape_validation.append(stats.sem(np.arctan(abs(actual_validate[indices_validation]-predicted_validation_nozero[indices_validation])/actual_validate[indices_validation])))

x_axis.append(actual_value+decimal.Decimal('0.05'))

actual_value += decimal.Decimal('0.1')
```


```
fig, ax1 = plt.subplots() fig.patch.set_facecolor('white') l1 = ax1.errorbar(x_axis, all_maape_training, all_errorbars_maape_training, fmt='.',color='cornflowerblue', label="Training") l2 = ax1.errorbar(x_axis, all_maape_validation, all_errorbars_maape_validation, fmt='.',color='g', label="Validation") ax1.set_ylabel('Mean Arctangent Absolute Percentage Error', fontsize=18) ax1.set_xlabel('3D MSD fourth-root', fontsize=18) plt.tick_params(axis='both', which='major', labelsize=14) plt.tick_params(axis='both', which='minor', labelsize=8) ax1.legend(loc='best', fontsize=14, numpoints=1) ax1.xaxis.set_ticks(np.arange(0, 1.1, 0.1)) #ax1.yaxis.set_ticks(np.arange(0, 1.1, 0.1)) #plt.ylim(0,0.3)#np.array([all_maape_training,all_maape_validation]).max()+0.005) #plt.xlim(0.1,1) plt.tight_layout()
```


#### False positives and negatives

```
def get_false_pos_neg(cutoff, metric_array, predicted_array):
```
-------------
false positive

```
indices_actual_above_cutoff = np.where(cutoff <= metric_array)[0]
indices_predicted_under_cutoff = np.where(predicted_array < cutoff)[0]
intersection_false_positive = np.intersect1d(indices_actual_above_cutoff, indices_predicted_under_cutoff)
false_positive = len(intersection_false_positive)/len(indices_actual_above_cutoff)

#false_negative
indices_actual_under_cutoff = np.where(cutoff >= metric_array)[0]
indices_predicted_above_cutoff = np.where(predicted_array > cutoff)[0]
intersection_false_negative = np.intersect1d(indices_actual_under_cutoff, indices_predicted_above_cutoff)
false_negative = len(intersection_false_negative)/len(indices_actual_under_cutoff)

correct = 1-false_positive-false_negative
return false_positive, false_negative, correct
```



```
x_axis = [] all_false_positive_ratios_val = [] all_false_negative_ratios_val = [] correct_val = []

if (len(Y_train) != len(predicted_training) or len(Y_validate) != len(predicted_validation)): raise ValueError("The arrays of actual values and predictions did not have the same shape!")

cutoff = decimal.Decimal('0.1') while (cutoff <= decimal.Decimal('0.9')): false_positive_ratio_val, false_negative_ratio_val, correct_ratio_val = get_false_pos_neg(cutoff, Y_validate, predicted_validation) all_false_positive_ratios_val.append(false_positive_ratio_val) all_false_negative_ratios_val.append(false_negative_ratio_val) x_axis.append(cutoff)
    
```
----------
```
cutoff += decimal.Decimal('0.01')
```

Plotting the above

```
fig, ax1 = plt.subplots() fig.patch.set_facecolor('white') #l1 = ax1.errorbar(x_axis, all_mape_training, all_errorbars_mape_training, color='cornflowerblue', label="Training") l2 = ax1.errorbar(x_axis, all_mape_validation, all_errorbars_mape_validation, color='g', label="Validation")

#l3 = ax1.scatter(x_axis, all_mape_training, s = 0.1np.array(datapoints_training), color='r') l4 = ax1.scatter(x_axis, all_mape_validation, s = 0.5np.array(datapoints_validation), color='r')

#Colour legend #fig.legend(handles=[l1,l2], loc=[0.1111,0.13], numpoints=1)

#Size legend handles, labels = l4.legend_elements(prop="sizes", num = [50,100,250,500,1000], alpha = 0.5, color='red') ax1.legend(handles, labels, loc="best", title="#Datapoints", numpoints = 1, labelspacing=1)

ax1.set_ylabel('Mean Absolute Percentage Error', fontsize=18) ax1.set_xlabel('3D MSD fourth-root', fontsize=18) plt.tick_params(axis='both', which='major', labelsize=15) plt.tick_params(axis='both', which='minor', labelsize=8) ax1.xaxis.set_ticks(np.arange(0, 1.1, 0.1)) #ax1.yaxis.set_ticks(np.arange(0, 1.1, 0.1)) plt.grid() plt.ylim(0,0.6)#np.array([all_mean_deviations_training,all_mean_deviations_validation]).max()+0.005) plt.xlim(0,1) plt.tight_layout()
```

```
fig, ax1 = plt.subplots() fig.patch.set_facecolor('white') l1 = ax1.scatter(x_axis, all_false_negative_ratios_val, marker='x', s=100, color='cornflowerblue', label="Negative", linewidth=3) l2 = ax1.scatter(x_axis, all_false_positive_ratios_val, marker='x', s=100, color='g', label="Positive", linewidth=3) #ax1.set_ylabel('False count ratio', fontsize=18) #ax1.set_xlabel('3D MSD fourth-root', fontsize=18) plt.tick_params(axis='both', which='major', labelsize=25) plt.tick_params(axis='both', which='minor', labelsize=8) ax1.legend(loc='best', fontsize=14, scatterpoints=1) ax1.xaxis.tick_top() ax1.yaxis.tick_right() plt.grid() #ax1.xaxis.set_ticks(np.arange(0, 1.1, 0.1)) #ax1.yaxis.set_ticks(np.arange(0, 1.1, 0.1)) plt.ylim(0.025,0.23)#np.array([all_false_negative_ratios_val, all_false_positive_ratios_val]).max()+0.005) plt.xlim(0.35,0.55) plt.tight_layout()
```


### Occlusion Maps

In [ ]:
def iter_occlusion(image, size=1):
    #this is where you choose the size of the patch
    #occlusion = np.full((size, size, 1), [0.5], np.float32)
    occlusion_center = np.full((size, size, 1), [0.5], np.float32)
    occlusion_padding = size 
    # print('padding...')
    #np.pad(input array, Number of values padded to the edges of each axis, pads with constant value)
    image_padded = np.pad(image, ( \
                        (occlusion_padding, occlusion_padding), (occlusion_padding, occlusion_padding), (0, 0) \
                        ), 'constant', constant_values = 0.0)

    for y in range(occlusion_padding, image.shape[0] + occlusion_padding, size): #range(start, stop, step)

        for x in range(occlusion_padding, image.shape[1] + occlusion_padding, size): #range(start, stop, step)
            tmp = image_padded.copy()
            
            #: means values between these values, y - occlusion_padding:y = y 
            #for size = 10, x - occlusion_padding:y = 20
            #tmp[y - occlusion_padding:y + occlusion_center.shape[0] + occlusion_padding - 4*occlusion_size, \
            #    x - occlusion_padding:x + occlusion_center.shape[1] + occlusion_padding - 4*occlusion_size] \
            #    = occlusion

            tmp[y:y + occlusion_center.shape[0], x:x + occlusion_center.shape[1]] = occlusion_center
            
            
            yield x - occlusion_padding, y - occlusion_padding, \
                  tmp[occlusion_padding:tmp.shape[0] - occlusion_padding, occlusion_padding:tmp.shape[1] \
                      - occlusion_padding]

In [ ]:
deviation = np.abs(Y_validate - predicted_metric_validation[:,0])
max_deviation_slice = np.where(deviation == np.max(deviation))[0][0]
min_deviation_slice = np.where(deviation == np.min(deviation))[0][0]

In [ ]:
prediction_max_bkground = model.predict((X_validate[max_deviation_slice]+300).reshape(1,512,512,4))
prediction_min_bkground = model.predict((X_validate[min_deviation_slice]+300).reshape(1,512,512,4))

USE THIS TO WRITE OUT TO A FILE WHAT THE ACTUAL VALUES AND PREDICTIONS ARE

In [ ]:
occlusion = "Max deviation slice: "+str(max_deviation_slice)+". Actual value: "+str(Y_validate[max_deviation_slice])+". \
Prediction: "+str(predicted_metric_validation[max_deviation_slice][0])+". Prediction background \
300: "+str(prediction_max_bkground[0][0])+". \nMin deviation Slice: "+str(min_deviation_slice)+" Actual value: \
"+str(Y_validate[min_deviation_slice])+". Prediction: "+str(predicted_metric_validation[min_deviation_slice][0])+". Prediction\
 background 300: "+str(prediction_min_bkground[0][0])+"."
with open(output_folder+'occlusion.txt', 'w') as f:
    f.write(occlusion)

`for i in range(1000): print(str(i) + "--> " + str(Y_validate[i]) + " " + str(predicted_metric_validation[i][0]))`

Define the image to be occluded

In [ ]:
i = max_deviation_slice
data = X_validate[i]+300

# Define the corresponding data
actual_metric = Y_validate[i]
predicted_metric = predicted_metric_validation[i][0]

print('Actual value of metric: ', actual_metric)
print('Unoccluded prediction from predicted_metric_validation: ', predicted_metric)

# input tensor for model.predict
#for multiple_patient_image_array
inp = data.reshape(1, 512, 512, 4)

prediction_i = model.predict(inp)
print('Unoccluded prediction from predicting on X_validate[i]+300: ', prediction_i[0,0])

# occlusion data
img_size = inp.shape[1]
occlusion_size = 4

In [ ]:
plt.imshow(data[:,:,0].astype(np.float64), cmap='magma')
plt.colorbar()
plt.savefig(output_folder+"i="+str(i)+"_max_deviation_300bkg.png")

In [ ]:
print('occluding...')

heatmap = np.zeros((512, 512), np.float64)

start=time.time()
for n, (x, y, img_float) in enumerate(iter_occlusion(data, size=occlusion_size)):
    
    X = img_float.reshape(1, 512, 512, 4)
    out = model.predict(X)
    print('{}: Occluded prediction: {}'.format(n, out[0,0]))
    print('x {} - {} | y {} - {}'.format(x, x + occlusion_size, y, y + occlusion_size))

    heatmap[y:y + occlusion_size, x:x + occlusion_size] = out[0,0]

stop = time.time()

print("These occlusions took: " + str(stop-start) + " seconds." )

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig = plt.figure(figsize=(15, 15))

ax1 = plt.subplot(1, 2, 1, aspect='equal')
hm = ax1.imshow(heatmap, cmap='viridis', origin='lower')
ax1.set_xlabel('x', fontsize=18)
ax1.set_ylabel('y', fontsize=18)

divider = make_axes_locatable(ax1)
cax1 = divider.append_axes("right", size="5%", pad=0.2)
cbar1 = plt.colorbar(hm, cax=cax1, format = '%1.6f') #CHECK THIS FOR EACH PLOT
cbar1.ax.set_ylabel(metric)

fig.tight_layout()
plt.savefig(output_folder + 'occlusion_map_'+metric+'_' + 'i:'+str(i) + '_' + organ + '_max_deviation_300bkg.png')
plt.show()

#### Create a heat map of window importance by subtracting overall predicted from predicted for each square

In [ ]:
#ground_truth = predicted_metric_validation[i]
ground_truth = prediction_i[0]

In [ ]:
fig = plt.figure(figsize=(15, 15))

#occlusion_data = np.rot90(heatmap-predicted_metric_validation[i], k=1, axes=(0,1))
ax1 = plt.subplot(1, 2, 1, aspect='equal')
maxDiff = np.amax(abs(heatmap-ground_truth))

hm = ax1.imshow(heatmap-ground_truth, cmap='Spectral', origin='lower', vmin = -maxDiff, vmax =maxDiff)

ax1.set_xlabel('x', fontsize=18)
ax1.set_ylabel('y', fontsize=18)
ax1.tick_params(axis='both', which='major', labelsize=18)

divider = make_axes_locatable(ax1)
cax1 = divider.append_axes("right", size="5%", pad=0.2)
cbar1 = plt.colorbar(hm, cax=cax1)
cbar1.ax.set_ylabel('Change in '+metric+' Due to Occlusion', fontsize =18)
cbar1.ax.tick_params(axis='both', which='major', labelsize=18)

fig.tight_layout()
plt.savefig(output_folder + 'occlusion_map_'+metric+'_change_' + 'i:'+str(i) + '_' + organ + '_max_deviation_300bkg.png')
plt.show()

#### Now the min deviation slice

In [ ]:
# Define the image to be occluded
i = min_deviation_slice
data = X_validate[i]+300

# Define the corresponding data
actual_metric = Y_validate[i]
predicted_metric = predicted_metric_validation[i][0]

print('Actual value of metric: ', actual_metric)
print('Unoccluded prediction from predicted_metric_validation: ', predicted_metric)

# input tensor for model.predict
#for multiple_patient_image_array
inp = data.reshape(1, 512, 512, 4)

prediction_i = model.predict(inp)
print('Unoccluded prediction from predicting on X_validate[i]+300: ', prediction_i[0,0])

# occlusion data
img_size = inp.shape[1]
occlusion_size = 4

In [ ]:
plt.imshow(data[:,:,0].astype(np.float64), cmap='magma')
plt.colorbar()
plt.savefig(output_folder+"i="+str(i)+"_min_deviation_300bkg.png")

In [ ]:
print('occluding...')

heatmap = np.zeros((512, 512), np.float64)

start=time.time()
for n, (x, y, img_float) in enumerate(iter_occlusion(data, size=occlusion_size)):
    
    X = img_float.reshape(1, 512, 512, 4)
    out = model.predict(X)
    print('{}: Occluded prediction: {}'.format(n, out[0,0]))
    print('x {} - {} | y {} - {}'.format(x, x + occlusion_size, y, y + occlusion_size))

    heatmap[y:y + occlusion_size, x:x + occlusion_size] = out[0,0]

stop = time.time()

print("These occlusions took: " + str(stop-start) + " seconds." )

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig = plt.figure(figsize=(15, 15))

ax1 = plt.subplot(1, 2, 1, aspect='equal')
hm = ax1.imshow(heatmap, cmap='viridis', origin='lower')
ax1.set_xlabel('x', fontsize=18)
ax1.set_ylabel('y', fontsize=18)

divider = make_axes_locatable(ax1)
cax1 = divider.append_axes("right", size="5%", pad=0.2)
cbar1 = plt.colorbar(hm, cax=cax1, format = '%1.6f') #CHECK THIS FOR EACH PLOT
cbar1.ax.set_ylabel(metric)

fig.tight_layout()
plt.savefig(output_folder + 'occlusion_map_'+metric+'_' + 'i:'+str(i) + '_' + organ + '_min_deviation_300bkg.png')
plt.show()

In [ ]:
#ground_truth = predicted_metric_validation[i]
ground_truth = prediction_i[0]

In [ ]:
fig = plt.figure(figsize=(15, 15))

#occlusion_data = np.rot90(heatmap-predicted_metric_validation[i], k=1, axes=(0,1))
ax1 = plt.subplot(1, 2, 1, aspect='equal')
maxDiff = np.amax(abs(heatmap-ground_truth))

hm = ax1.imshow(heatmap-ground_truth, cmap='Spectral', origin='lower', vmin = -maxDiff, vmax =maxDiff)

ax1.set_xlabel('x', fontsize=18)
ax1.set_ylabel('y', fontsize=18)
ax1.tick_params(axis='both', which='major', labelsize=18)

divider = make_axes_locatable(ax1)
cax1 = divider.append_axes("right", size="5%", pad=0.2)
cbar1 = plt.colorbar(hm, cax=cax1)
cbar1.ax.set_ylabel('Change in '+metric+' Due to Occlusion', fontsize =18)
cbar1.ax.tick_params(axis='both', which='major', labelsize=18)

fig.tight_layout()
plt.savefig(output_folder + 'occlusion_map_'+metric+'_change_' + 'i:'+str(i) + '_' + organ + '_min_deviation_300bkg.png')
plt.show()

## DONE